In [1]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.experimental.list_physical_devices())

import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

2.1.0
Num GPUs Available:  0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]


In [2]:
from muzero.config import MuZeroConfig
from muzero.main import Muzero

## TPU Setup

In [3]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
import gym
e = gym.make('CartPole-v0')

In [ ]:
print(e.observation_space) # Cart Position, Cart Velocity, Pole Angle, Velocity at Tip
print(e.action_space) # Push Cart Left/Right

### MuZero Config Params

In [ ]:
gpu_count = len(tf.config.experimental.list_physical_devices('GPU'))

def make_atari_config() -> MuZeroConfig:

    def visit_softmax_temperature(num_moves, training_steps):
        if training_steps < 500e3:
            return 1.0
        elif training_steps < 750e3:
            return 0.5
        else:
            return 0.25

    return MuZeroConfig(
        gym_env_name='CartPole-v0',
        action_space_size=2,
        selfplay_iterations=4, # Todo: implement None for continuous play
        max_moves=200,#000,  # Half an hour at action repeat 4.
        discount=0.997,
        dirichlet_alpha=0.25,
        num_simulations=50,
        batch_size=1024,#24,#1024,
        td_steps=10,
        num_actors=1,#350,
        lr_init=0.05,
        lr_decay_steps=350e3,
        visit_softmax_temperature_fn=visit_softmax_temperature,
        num_train_gpus=gpu_count)

### Testing

In [ ]:
import ray
ray.init(local_mode=False)

In [8]:
config = make_atari_config()
mz = Muzero(config)

In [ ]:
mz.run()

(pid=28159) 2020-05-24 10:55:32.487732: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=28159) 2020-05-24 10:55:32.504844: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff7688a48f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
(pid=28159) 2020-05-24 10:55:32.504878: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Self-play iter:   0%|          | 0/4 [00:00<?, ?it/s]


2020-05-24 10:55:37,576	ERROR worker.py:1030 -- Possible unhandled error from worker: ray::SharedStorage.latest_network() (pid=28159, ip=192.168.0.85)
  File "python/ray/_raylet.pyx", line 477, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 478, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1142, in ray._raylet.CoreWorker.store_task_outputs
  File "/Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/ray/serialization.py", line 401, in serialize
    return self._serialize_to_msgpack(metadata, value)
  File "/Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/ray/serialization.py", line 373, in _serialize_to_msgpack
    self._serialize_to_pickle5(metadata, python_objects)
  File "/Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/ray/serialization.py", line 353, in _serialize_to_pickle5
    raise e
  File "/Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/si

In [10]:
# model = mz.storage.latest_network()

In [ ]:
# game = ray.get(mz.replay_buffer.sample_game.remote())

In [17]:
mz.simulate()

In [10]:
# import ray
ray.shutdown()

In [ ]:
# !pip install memory-profiler

**ncalls**
for the number of calls,

**tottime**
for the total time spent in the given function (and excluding time made in calls to sub-functions)

**percall**
is the quotient of tottime divided by ncalls

**cumtime**
is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.

**percall**
is the quotient of cumtime divided by primitive calls

**filename:lineno(function)**
provides the respective data of each function

In [28]:
import cProfile
cProfile.run('run_selfplay(config, ss, rbuf)', filename='trace.prof')

Self-play iter: 100%|██████████| 1/1 [02:28<00:00, 148.28s/it]


In [ ]:
import pstats
p = pstats.Stats('trace.prof')
p.sort_stats('tottime').print_stats(10)

In [48]:
#@title Default title text
trace="""ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       47    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(all)
      294    0.001    0.000    0.011    0.000 <__array_function__ internals>:2(amax)
      338    0.001    0.000    0.008    0.000 <__array_function__ internals>:2(any)
      297    0.001    0.000    0.017    0.000 <__array_function__ internals>:2(concatenate)
        2    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(copyto)
      370    0.001    0.000    0.009    0.000 <__array_function__ internals>:2(prod)
      294    0.001    0.000    0.027    0.000 <__array_function__ internals>:2(stack)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:103(release)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:147(__enter__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:151(__exit__)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:194(_lock_unlock_module)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:211(_call_with_frames_removed)
        6    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:222(_verbose_message)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:35(_new_module)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:369(__init__)
      190    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:391(__eq__)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:403(cached)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:424(has_location)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:504(_init_module_attrs)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:564(module_from_spec)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:58(__init__)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:78(acquire)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:917(_sanity_check)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:966(_find_and_load)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:982(_gcd_import)
      129    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:106(_write_atomic)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:1359(_get_supported_file_loaders)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:263(cache_from_source)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:361(_get_cached)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:373(_calc_mode)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:393(_check_name_wrapper)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:47(_w_long)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:497(_code_to_bytecode)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:524(spec_from_file_location)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:57(_path_join)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:59(<listcomp>)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:63(_path_split)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:661(is_package)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:669(create_module)
        2    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:672(exec_module)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:735(source_to_code)
        2    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap_external>:743(get_code)
        6    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:75(_path_stat)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:800(__init__)
      190    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:806(__eq__)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:825(get_filename)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:830(get_data)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:840(path_stats)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:845(_cache_bytecode)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:85(_path_is_mode_type)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:850(set_data)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:99(_path_isdir)
    71628    0.252    0.000   24.637    0.000 <ipython-input-40-ef54d4819491>:13(update)
   102386    1.625    0.000   30.763    0.000 <ipython-input-40-ef54d4819491>:17(normalize)
      294    0.000    0.000    0.000    0.000 <ipython-input-40-ef54d4819491>:9(__init__)
        1    0.000    0.000    0.201    0.201 <ipython-input-41-b6219d23a42f>:56(new_game)
      294    0.000    0.000    0.000    0.000 <ipython-input-41-b6219d23a42f>:61(visit_softmax_temperature)
      882    0.000    0.000    0.000    0.000 <ipython-input-42-30c928fcc991>:12(__gt__)
   121128    0.042    0.000    0.042    0.000 <ipython-input-42-30c928fcc991>:3(__init__)
   259602    0.044    0.000    0.044    0.000 <ipython-input-42-30c928fcc991>:6(__hash__)
    77893    0.024    0.000    0.024    0.000 <ipython-input-42-30c928fcc991>:9(__eq__)
    71628    0.088    0.000    0.109    0.000 <ipython-input-43-28c03be0c141>:12(expanded)
   174308    0.529    0.000   18.052    0.000 <ipython-input-43-28c03be0c141>:15(value)
    60270    0.050    0.000    0.050    0.000 <ipython-input-43-28c03be0c141>:3(__init__)
    14700    0.027    0.000    0.070    0.000 <ipython-input-44-ef803af33755>:11(clone)
    56928    0.079    0.000    0.096    0.000 <ipython-input-44-ef803af33755>:14(add_action)
    14700    0.006    0.000    0.006    0.000 <ipython-input-44-ef803af33755>:17(last_action)
    14700    0.036    0.000    0.083    0.000 <ipython-input-44-ef803af33755>:20(action_space)
    14700    0.034    0.000    0.047    0.000 <ipython-input-44-ef803af33755>:21(<listcomp>)
    14994    0.043    0.000    0.043    0.000 <ipython-input-44-ef803af33755>:7(__init__)
      295    0.000    0.000    0.000    0.000 <ipython-input-45-8c7d8a00adce>:13(terminal)
      294    0.002    0.000    0.003    0.000 <ipython-input-45-8c7d8a00adce>:16(legal_actions)
      294    0.001    0.000    0.002    0.000 <ipython-input-45-8c7d8a00adce>:19(<listcomp>)
      295    0.025    0.000    0.035    0.000 <ipython-input-45-8c7d8a00adce>:21(prepro)
      294    0.001    0.000    0.001    0.000 <ipython-input-45-8c7d8a00adce>:29(get_obs)
        1    0.000    0.000    0.201    0.201 <ipython-input-45-8c7d8a00adce>:3(__init__)
      294    0.003    0.000    0.254    0.001 <ipython-input-45-8c7d8a00adce>:8(step)
      295    0.001    0.000    0.001    0.000 <ipython-input-46-1362272aa653>:12(terminal)
      294    0.001    0.000    0.004    0.000 <ipython-input-46-1362272aa653>:15(legal_actions)
      294    0.002    0.000    0.256    0.001 <ipython-input-46-1362272aa653>:18(apply)
      294    0.003    0.000    0.049    0.000 <ipython-input-46-1362272aa653>:23(store_search_statistics)
     1470    0.000    0.000    0.000    0.000 <ipython-input-46-1362272aa653>:25(<genexpr>)
     1470    0.001    0.000    0.001    0.000 <ipython-input-46-1362272aa653>:26(<genexpr>)
      294    0.002    0.000    0.005    0.000 <ipython-input-46-1362272aa653>:29(<listcomp>)
        1    0.000    0.000    0.201    0.201 <ipython-input-46-1362272aa653>:3(__init__)
      294    0.006    0.000    0.200    0.001 <ipython-input-46-1362272aa653>:35(make_image)
      294    0.001    0.000    0.001    0.000 <ipython-input-46-1362272aa653>:97(action_history)
        1    0.000    0.000    0.000    0.000 <ipython-input-47-5f58c4392d1b>:8(save_game)
       10    0.000    0.000    2.271    0.227 <ipython-input-48-a3aee8082806>:14(ConvBlock)
        1    0.000    0.000    0.221    0.221 <ipython-input-48-a3aee8082806>:20(ReprNet)
        1    0.000    0.000    1.955    1.955 <ipython-input-48-a3aee8082806>:28(DynaNet)
        1    0.000    0.000    0.149    0.149 <ipython-input-48-a3aee8082806>:39(PredNet)
      294    0.008    0.000    0.754    0.003 <ipython-input-53-ed847ba989f4>:16(initial_inference)
      294    0.004    0.000    0.005    0.000 <ipython-input-53-ed847ba989f4>:22(<dictcomp>)
        1    0.000    0.000    2.325    2.325 <ipython-input-53-ed847ba989f4>:3(__init__)
    14700    0.421    0.000   27.850    0.002 <ipython-input-53-ed847ba989f4>:32(recurrent_inference)
    14700    0.188    0.000    0.230    0.000 <ipython-input-53-ed847ba989f4>:40(<dictcomp>)
      294    0.000    0.000    0.000    0.000 <ipython-input-53-ed847ba989f4>:48(training_steps)
        1    0.000    0.000    2.325    2.325 <ipython-input-54-0886533d695a>:16(make_uniform_network)
        1    0.000    0.000    2.325    2.325 <ipython-input-54-0886533d695a>:6(latest_network)
        1    0.427    0.427  190.490  190.490 <ipython-input-55-37c85f058a24>:1(run_selfplay)
        1    0.086    0.086  181.856  181.856 <ipython-input-56-aaae3305ad34>:1(play_game)
      294    0.004    0.000    0.021    0.000 <ipython-input-57-fd1ed6f6ed97>:1(add_exploration_noise)
      294    0.003    0.000    0.074    0.000 <ipython-input-58-4ec0544e5e1c>:1(select_action)
      294    0.008    0.000    0.070    0.000 <ipython-input-58-4ec0544e5e1c>:14(softmax_sample)
      294    0.000    0.000    0.000    0.000 <ipython-input-58-4ec0544e5e1c>:15(<listcomp>)
      294    0.000    0.000    0.000    0.000 <ipython-input-58-4ec0544e5e1c>:16(<listcomp>)
      294    0.006    0.000    0.020    0.000 <ipython-input-58-4ec0544e5e1c>:22(softmax)
      294    0.001    0.000    0.001    0.000 <ipython-input-58-4ec0544e5e1c>:6(<listcomp>)
      294    0.561    0.002  180.199    0.613 <ipython-input-59-6a2401fc79df>:1(run_mcts)
    56928    0.192    0.000  107.099    0.002 <ipython-input-60-c9d2d71ff63b>:1(select_child)
   227712    2.436    0.000   67.272    0.000 <ipython-input-60-c9d2d71ff63b>:10(ucb_score)
   284640    0.616    0.000   67.888    0.000 <ipython-input-60-c9d2d71ff63b>:4(<genexpr>)
    14994    0.118    0.000    0.358    0.000 <ipython-input-61-5f8fc43a9ebe>:1(expand_node)
    14994    0.089    0.000    0.158    0.000 <ipython-input-61-5f8fc43a9ebe>:6(<dictcomp>)
    14700    1.077    0.000   43.962    0.003 <ipython-input-62-24dec825edc5>:1(backpropagate)
        1    0.000    0.000  190.543  190.543 <string>:1(<module>)
     1049    0.000    0.000    0.001    0.000 <string>:12(__new__)
      128    0.000    0.000    0.000    0.000 <string>:16(_make)
       18    0.000    0.000    0.000    0.000 <string>:24(_replace)
        1    0.000    0.000    0.000    0.000 __init__.py:108(import_module)
        2    0.000    0.000    0.001    0.001 __init__.py:11(unparse)
        1    0.000    0.000    0.000    0.000 __init__.py:120(getLevelName)
        1    0.000    0.000    0.004    0.004 __init__.py:1310(warning)
       36    0.000    0.000    0.000    0.000 __init__.py:1359(log)
        1    0.000    0.000    0.000    0.000 __init__.py:1406(makeRecord)
        1    0.000    0.000    0.004    0.004 __init__.py:1421(_log)
        1    0.000    0.000    0.004    0.004 __init__.py:1446(handle)
        1    0.000    0.000    0.004    0.004 __init__.py:1500(callHandlers)
       37    0.000    0.000    0.000    0.000 __init__.py:1530(getEffectiveLevel)
       37    0.000    0.000    0.000    0.000 __init__.py:1544(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:251(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:329(getMessage)
        1    0.000    0.000    0.000    0.000 __init__.py:387(usesTime)
        1    0.000    0.000    0.000    0.000 __init__.py:390(format)
        1    0.000    0.000    0.000    0.000 __init__.py:4(_game_dir)
        1    0.000    0.000    0.000    0.000 __init__.py:458(close)
        3    0.000    0.000    0.000    0.000 __init__.py:517(__init__)
       38    0.000    0.000    0.000    0.000 __init__.py:537(__missing__)
        1    0.000    0.000    0.000    0.000 __init__.py:542(usesTime)
        1    0.000    0.000    0.000    0.000 __init__.py:548(formatMessage)
        1    0.000    0.000    0.000    0.000 __init__.py:564(format)
        3    0.000    0.000    0.000    0.000 __init__.py:586(update)
        1    0.000    0.000    0.000    0.000 __init__.py:639(__del__)
        1    0.000    0.000    0.000    0.000 __init__.py:7(get_game_path)
        2    0.000    0.000    0.000    0.000 __init__.py:705(filter)
        2    0.000    0.000    0.000    0.000 __init__.py:809(acquire)
        2    0.000    0.000    0.000    0.000 __init__.py:816(release)
        1    0.000    0.000    0.000    0.000 __init__.py:829(format)
        1    0.000    0.000    0.004    0.004 __init__.py:852(handle)
        1    0.000    0.000    0.004    0.004 __init__.py:971(flush)
        1    0.000    0.000    0.004    0.004 __init__.py:982(emit)
       23    0.000    0.000    0.000    0.000 _asarray.py:16(asarray)
      294    0.000    0.000    0.001    0.000 _asarray.py:88(asanyarray)
      926    0.001    0.000    0.004    0.000 _collections_abc.py:657(get)
      656    0.001    0.000    0.002    0.000 _collections_abc.py:664(__contains__)
     2484    0.003    0.000    0.004    0.000 _collections_abc.py:676(items)
     2484    0.001    0.000    0.001    0.000 _collections_abc.py:698(__init__)
     7585    0.016    0.000    0.016    0.000 _collections_abc.py:742(__iter__)
      336    0.000    0.000    0.001    0.000 _collections_abc.py:879(__iter__)
       20    0.000    0.000    0.000    0.000 _internal.py:830(npy_ctypes_check)
      294    0.000    0.000    0.002    0.000 _methods.py:36(_sum)
        3    0.000    0.000    0.000    0.000 _methods.py:44(_any)
        1    0.000    0.000    0.000    0.000 _monitor.py:98(report)
        6    0.000    0.000    0.000    0.000 _ufunc_config.py:139(geterr)
        6    0.000    0.000    0.000    0.000 _ufunc_config.py:39(seterr)
        3    0.000    0.000    0.000    0.000 _ufunc_config.py:441(__enter__)
        3    0.000    0.000    0.000    0.000 _ufunc_config.py:446(__exit__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:106(remove)
      173    0.000    0.000    0.001    0.000 _weakrefset.py:116(update)
      909    0.000    0.000    0.000    0.000 _weakrefset.py:16(__init__)
      909    0.000    0.000    0.001    0.000 _weakrefset.py:20(__enter__)
      909    0.001    0.000    0.001    0.000 _weakrefset.py:26(__exit__)
      397    0.000    0.000    0.001    0.000 _weakrefset.py:36(__init__)
      227    0.000    0.000    0.000    0.000 _weakrefset.py:38(_remove)
      698    0.000    0.000    0.000    0.000 _weakrefset.py:52(_commit_removals)
     1231    0.001    0.000    0.003    0.000 _weakrefset.py:58(__iter__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:67(__len__)
   432634    0.450    0.000    0.450    0.000 _weakrefset.py:70(__contains__)
      351    0.389    0.001    1.716    0.005 _weakrefset.py:81(add)
   239505    0.548    0.000    0.998    0.000 abc.py:180(__instancecheck__)
      174    0.000    0.000    0.000    0.000 activations.py:338(linear)
       23    0.000    0.000    0.000    0.000 activations.py:425(get)
     18/6    0.000    0.000    0.000    0.000 activity.py:126(referenced)
      404    0.001    0.000    0.002    0.000 activity.py:172(finalize)
       55    0.000    0.000    0.000    0.000 activity.py:191(mark_param)
       44    0.000    0.000    0.000    0.000 activity.py:210(__init__)
       22    0.000    0.000    0.000    0.000 activity.py:224(__init__)
      170    0.000    0.000    0.000    0.000 activity.py:232(_in_constructor)
      710    0.002    0.000    0.006    0.000 activity.py:251(_track_symbol)
      404    0.000    0.000    0.003    0.000 activity.py:303(_enter_scope)
      404    0.000    0.000    0.002    0.000 activity.py:306(_exit_scope)
      374    0.000    0.000    0.003    0.000 activity.py:312(_exit_and_record_scope)
      173    0.000    0.000    0.035    0.000 activity.py:317(_process_statement)
        8    0.000    0.000    0.000    0.000 activity.py:334(visit_Expr)
        8    0.000    0.000    0.000    0.000 activity.py:337(visit_Raise)
       22    0.000    0.000    0.004    0.000 activity.py:340(visit_Return)
       95    0.000    0.000    0.022    0.000 activity.py:343(visit_Assign)
      540    0.000    0.000    0.012    0.000 activity.py:366(visit_Name)
      170    0.000    0.000    0.009    0.000 activity.py:384(visit_Attribute)
  139/107    0.000    0.000    0.021    0.000 activity.py:409(visit_Call)
       22    0.000    0.000    0.003    0.000 activity.py:477(visit_arguments)
       22    0.000    0.000    0.041    0.002 activity.py:499(visit_FunctionDef)
       18    0.000    0.000    0.027    0.002 activity.py:533(visit_With)
       18    0.000    0.000    0.006    0.000 activity.py:539(visit_withitem)
        8    0.000    0.000    0.001    0.000 activity.py:581(visit_ExceptHandler)
       22    0.000    0.000    0.042    0.002 activity.py:592(resolve)
      426    0.001    0.000    0.003    0.000 activity.py:92(__init__)
       10    0.000    0.000    0.004    0.000 advanced_activations.py:344(__init__)
      118    0.000    0.000    0.063    0.001 advanced_activations.py:360(call)
      146    0.000    0.000    0.000    0.000 ag_ctx.py:29(_control_ctx)
       38    0.000    0.000    0.000    0.000 ag_ctx.py:35(control_status_ctx)
       36    0.000    0.000    0.000    0.000 ag_ctx.py:49(__init__)
       36    0.000    0.000    0.000    0.000 ag_ctx.py:53(__enter__)
       36    0.000    0.000    0.000    0.000 ag_ctx.py:61(__exit__)
      270    0.000    0.000    0.002    0.000 ag_logging.py:114(get_verbosity)
      270    0.000    0.000    0.002    0.000 ag_logging.py:121(has_verbosity)
      231    0.000    0.000    0.002    0.000 ag_logging.py:138(log)
        3    0.000    0.000    0.000    0.000 ale_python_interface.py:115(_as_bytes)
        1    0.000    0.000    0.000    0.000 ale_python_interface.py:127(__init__)
        1    0.000    0.000    0.000    0.000 ale_python_interface.py:141(setInt)
        1    0.000    0.000    0.000    0.000 ale_python_interface.py:145(setFloat)
        1    0.188    0.188    0.188    0.188 ale_python_interface.py:148(loadROM)
      880    0.152    0.000    0.152    0.000 ale_python_interface.py:151(act)
      294    0.001    0.000    0.001    0.000 ale_python_interface.py:154(game_over)
        1    0.010    0.010    0.010    0.010 ale_python_interface.py:157(reset_game)
        1    0.000    0.000    0.000    0.000 ale_python_interface.py:196(getMinimalActionSet)
      294    0.000    0.000    0.000    0.000 ale_python_interface.py:205(lives)
        1    0.000    0.000    0.000    0.000 ale_python_interface.py:211(getScreenDims)
      295    0.023    0.000    0.050    0.000 ale_python_interface.py:251(getScreenRGB2)
        3    0.000    0.000    0.000    0.000 ale_python_interface.py:353(__del__)
     7010    0.004    0.000    0.008    0.000 anno.py:111(getanno)
    28849    0.014    0.000    0.029    0.000 anno.py:119(hasanno)
     3155    0.004    0.000    0.006    0.000 anno.py:123(setanno)
     5010    0.002    0.000    0.007    0.000 anno.py:141(copyanno)
        2    0.000    0.000    0.001    0.000 anno.py:150(dup)
       35    0.000    0.000    0.000    0.000 api.py:126(__init__)
     1835    0.001    0.000    0.001    0.000 api.py:129(get_effective_source_map)
       35    0.000    0.000    0.000    0.000 api.py:155(autograph_artifact)
       38    0.000    0.000    0.000    0.000 api.py:160(is_autograph_artifact)
       53    0.000    0.000    1.279    0.024 api.py:337(_call_unconverted)
      114    0.000    0.000    0.000    0.000 api.py:351(_is_known_loaded_type)
    88/18    0.002    0.000    1.561    0.087 api.py:378(converted_call)
      228    0.001    0.000    0.002    0.000 api.py:480(<genexpr>)
        2    0.000    0.000    0.000    0.000 arg_defaults.py:72(visit_FunctionDef)
        2    0.000    0.000    0.000    0.000 arg_defaults.py:79(visit_arguments)
        2    0.000    0.000    0.000    0.000 arg_defaults.py:94(transform)
    44688    0.095    0.000    1.588    0.000 array_ops.py:1041(strided_slice)
   134064    0.106    0.000    2.791    0.000 array_ops.py:1281(stack)
   134388    0.158    0.000    0.583    0.000 array_ops.py:1432(_should_not_autopack)
   270398    0.085    0.000    0.085    0.000 array_ops.py:1438(<genexpr>)
   134388    0.067    0.000    0.650    0.000 array_ops.py:1442(_autopacking_conversion_function)
       43    0.000    0.000    0.003    0.000 array_ops.py:198(fill)
      226    0.001    0.000    0.107    0.000 array_ops.py:239(identity)
       23    0.000    0.000    0.003    0.000 array_ops.py:2676(wrapped)
       23    0.000    0.000    0.003    0.000 array_ops.py:2684(zeros)
       20    0.000    0.000    0.003    0.000 array_ops.py:2934(ones)
        3    0.000    0.000    0.003    0.001 array_ops.py:2984(placeholder)
       10    0.000    0.000    0.010    0.001 array_ops.py:3029(placeholder_with_default)
      311    0.001    0.000    0.164    0.001 array_ops.py:3144(pad_v2)
      311    0.003    0.000    0.163    0.001 array_ops.py:3202(pad)
   221/17    0.000    0.000    0.000    0.000 array_ops.py:3297(_get_paddings_constant)
    85/17    0.000    0.000    0.000    0.000 array_ops.py:3314(<listcomp>)
      294    0.001    0.000    0.011    0.000 array_ops.py:347(expand_dims_v2)
       20    0.000    0.000    0.011    0.001 array_ops.py:4093(squeeze)
       20    0.000    0.000    0.013    0.001 array_ops.py:4148(squeeze_v2)
       37    0.000    0.000    0.023    0.001 array_ops.py:58(reshape)
    44688    0.073    0.000    0.227    0.000 array_ops.py:823(_check_index)
    44688    0.787    0.000    6.693    0.000 array_ops.py:843(_slice_helper)
      385    0.000    0.000    0.001    0.000 ast.py:118(copy_location)
    57878    0.015    0.000    0.020    0.000 ast.py:166(iter_fields)
      256    0.000    0.000    0.000    0.000 ast.py:178(iter_child_nodes)
      131    0.000    0.000    0.001    0.000 ast.py:215(walk)
18632/170    0.013    0.000    0.231    0.001 ast.py:249(visit)
 2676/149    0.005    0.000    0.014    0.000 ast.py:255(generic_visit)
       35    0.000    0.000    0.002    0.000 ast.py:30(parse)
14149/335    0.033    0.000    0.177    0.001 ast.py:302(generic_visit)
       35    0.000    0.000    0.011    0.000 ast3.py:10(visit_Module)
    43/38    0.000    0.000    0.004    0.000 ast3.py:114(visit_Call)
       34    0.000    0.000    0.000    0.000 ast3.py:139(visit_NameConstant)
       12    0.000    0.000    0.001    0.000 ast3.py:149(visit_arguments)
      213    0.000    0.000    0.003    0.000 ast3.py:162(visit_Name)
       16    0.000    0.000    0.000    0.000 ast3.py:17(visit_Num)
       22    0.000    0.000    0.000    0.000 ast3.py:172(visit_arg)
        4    0.000    0.000    0.000    0.000 ast3.py:187(visit_ExceptHandler)
       29    0.000    0.000    0.000    0.000 ast3.py:218(visit_Constant)
       23    0.000    0.000    0.000    0.000 ast3.py:234(_make_arg)
       83    0.000    0.000    0.001    0.000 ast3.py:250(visit_Name)
        2    0.000    0.000    0.000    0.000 ast3.py:258(visit_ExceptHandler)
      6/2    0.000    0.000    0.002    0.001 ast3.py:310(visit_FunctionDef)
        4    0.000    0.000    0.000    0.000 ast3.py:34(visit_Str)
        2    0.000    0.000    0.001    0.001 ast3.py:353(visit_With)
    19/15    0.000    0.000    0.001    0.000 ast3.py:369(visit_Call)
        6    0.000    0.000    0.000    0.000 ast3.py:378(visit_arguments)
        6    0.000    0.000    0.000    0.000 ast3.py:380(<listcomp>)
        6    0.000    0.000    0.000    0.000 ast3.py:392(<listcomp>)
       35    0.000    0.000    0.011    0.000 ast3.py:398(ast_to_gast)
        2    0.000    0.000    0.002    0.001 ast3.py:402(gast_to_ast)
     12/6    0.000    0.000    0.006    0.001 ast3.py:50(visit_FunctionDef)
        4    0.000    0.000    0.003    0.001 ast3.py:96(visit_With)
      395    0.002    0.000    0.003    0.000 ast_util.py:278(parallel_walk)
       82    0.000    0.000    0.000    0.000 ast_util.py:34(__init__)
  2017/82    0.006    0.000    0.017    0.000 ast_util.py:38(copy)
   228/41    0.000    0.000    0.014    0.000 ast_util.py:42(<listcomp>)
        5    0.000    0.000    0.006    0.001 ast_util.py:44(<genexpr>)
       82    0.000    0.000    0.017    0.000 ast_util.py:64(copy_clean)
   476/43    0.000    0.000    0.013    0.000 astn.py:11(<listcomp>)
  876/180    0.002    0.000    0.011    0.000 astn.py:17(generic_visit)
  2710/45    0.002    0.000    0.013    0.000 astn.py:9(_visit)
      294    0.004    0.000    0.214    0.001 atari_env.py:111(step)
      295    0.001    0.000    0.051    0.000 atari_env.py:125(_get_image)
      295    0.001    0.000    0.052    0.000 atari_env.py:135(_get_obs)
        1    0.000    0.000    0.010    0.010 atari_env.py:143(reset)
        1    0.000    0.000    0.191    0.191 atari_env.py:26(__init__)
        1    0.000    0.000    0.188    0.188 atari_env.py:83(seed)
     4962    0.003    0.000    0.008    0.000 auto_control_deps.py:121(op_is_stateful)
       93    0.000    0.000    0.000    0.000 auto_control_deps.py:148(__init__)
      186    0.001    0.000    0.091    0.000 auto_control_deps.py:152(mark_as_return)
       93    0.000    0.000    0.002    0.000 auto_control_deps.py:195(__enter__)
       93    0.018    0.000    0.284    0.003 auto_control_deps.py:270(__exit__)
     2481    0.000    0.000    0.000    0.000 auto_control_deps.py:391(<listcomp>)
       10    0.000    0.000    0.000    0.000 auto_control_deps.py:401(<listcomp>)
     2481    0.012    0.000    0.241    0.000 auto_control_deps.py:452(_get_resource_inputs)
     2481    0.002    0.000    0.003    0.000 auto_control_deps.py:471(<listcomp>)
     2481    0.001    0.000    0.002    0.000 auto_control_deps.py:472(<listcomp>)
     1484    0.001    0.000    0.003    0.000 auto_control_deps_utils.py:102(<genexpr>)
     1719    0.002    0.000    0.004    0.000 auto_control_deps_utils.py:109(<genexpr>)
       20    0.000    0.000    0.002    0.000 auto_control_deps_utils.py:33(get_read_only_resource_input_indices_graph)
       80    0.000    0.000    0.000    0.000 auto_control_deps_utils.py:51(<listcomp>)
       80    0.000    0.000    0.000    0.000 auto_control_deps_utils.py:60(_get_read_only_resource_input_indices_op)
       80    0.000    0.000    0.000    0.000 auto_control_deps_utils.py:63(<listcomp>)
     2481    0.011    0.000    0.186    0.000 auto_control_deps_utils.py:86(get_read_write_resource_inputs)
        3    0.000    0.000    0.004    0.001 backend.py:1024(placeholder)
       36    0.000    0.000    0.000    0.000 backend.py:1091(is_placeholder)
       10    0.000    0.000    0.011    0.001 backend.py:1110(freezable_variable)
       84    0.000    0.000    0.001    0.000 backend.py:1181(int_shape)
       20    0.000    0.000    0.000    0.000 backend.py:169(cast_to_floatx)
        3    0.000    0.000    0.000    0.000 backend.py:207(get_uid)
       71    0.000    0.000    0.005    0.000 backend.py:304(learning_phase)
       71    0.000    0.000    0.000    0.000 backend.py:334(global_learning_phase_is_set)
       71    0.000    0.000    0.000    0.000 backend.py:338(_mark_func_graph_as_unsaveable)
      118    0.004    0.000    0.062    0.001 backend.py:4400(relu)
     1989    0.002    0.000    0.007    0.000 backend.py:531(get_graph)
       38    0.000    0.000    0.000    0.000 backend.py:598(get_default_graph_uid_map)
       71    0.000    0.000    0.000    0.000 backend.py:6123(_key)
       81    0.000    0.000    0.000    0.000 backend.py:6142(_get_recursive)
       81    0.000    0.000    0.000    0.000 backend.py:6155(__getitem__)
       56    0.000    0.000    0.000    0.000 backend.py:725(is_sparse)
      483    0.001    0.000    0.001    0.000 backend.py:776(name_scope)
       56    0.000    0.000    0.000    0.000 backend.py:862(track_variable)
       38    0.000    0.000    0.000    0.000 backend.py:870(unique_object_name)
       12    0.000    0.000    0.000    0.000 backend_config.py:112(image_data_format)
       61    0.000    0.000    0.000    0.000 backend_config.py:65(floatx)
     1856    0.002    0.000    0.004    0.000 backprop.py:165(_must_record_gradient)
2512/1932    0.005    0.000    0.040    0.000 base.py:452(_method_wrapper)
      310    0.000    0.000    0.001    0.000 base.py:576(_maybe_initialize_trackable)
      122    0.000    0.000    0.000    0.000 base.py:655(_deferred_dependencies)
      122    0.000    0.000    0.000    0.000 base.py:670(_lookup_dependency)
       56    0.001    0.000    0.061    0.001 base.py:684(_add_variable_with_custom_getter)
       56    0.000    0.000    0.000    0.000 base.py:756(_preload_simple_restoration)
      122    0.001    0.000    0.003    0.000 base.py:793(_track_trackable)
       66    0.000    0.000    0.001    0.000 base.py:847(_handle_deferred_dependencies)
      396    0.000    0.000    0.000    0.000 base_layer.py:1005(trainable)
      482    0.000    0.000    0.000    0.000 base_layer.py:1025(input_spec)
       38    0.000    0.000    0.000    0.000 base_layer.py:1060(input_spec)
   519/71    0.001    0.000    0.012    0.000 base_layer.py:1291(_clear_losses)
       20    0.000    0.000    0.006    0.000 base_layer.py:1380(add_update)
       20    0.000    0.000    0.004    0.000 base_layer.py:1433(process_update)
       20    0.000    0.000    0.004    0.000 base_layer.py:1459(<listcomp>)
       20    0.000    0.000    0.000    0.000 base_layer.py:1463(<listcomp>)
        3    0.000    0.000    0.000    0.000 base_layer.py:1792(input)
       41    0.000    0.000    0.000    0.000 base_layer.py:1916(inbound_nodes)
       35    0.000    0.000    0.000    0.000 base_layer.py:1922(outbound_nodes)
       41    0.000    0.000    0.002    0.000 base_layer.py:1983(_set_dtype_policy)
     1022    0.001    0.000    0.001    0.000 base_layer.py:2012(_compute_dtype)
      483    0.002    0.000    0.014    0.000 base_layer.py:2025(_maybe_cast_inputs)
      447    0.003    0.000    0.005    0.000 base_layer.py:2040(f)
      483    0.000    0.000    0.000    0.000 base_layer.py:2099(_name_scope)
       41    0.000    0.000    0.002    0.000 base_layer.py:2102(_init_set_name)
      483    0.000    0.000    0.000    0.000 base_layer.py:2201(_handle_activity_regularization)
      483    0.003    0.000    0.013    0.000 base_layer.py:2218(_set_mask_metadata)
      894    0.000    0.000    0.001    0.000 base_layer.py:2223(<genexpr>)
       36    0.000    0.000    0.000    0.000 base_layer.py:2233(<listcomp>)
      273    0.000    0.000    0.000    0.000 base_layer.py:2235(<listcomp>)
      174    0.000    0.000    0.000    0.000 base_layer.py:2241(<listcomp>)
      483    0.002    0.000    0.012    0.000 base_layer.py:2258(_collect_input_masks)
      483    0.000    0.000    0.000    0.000 base_layer.py:2266(<lambda>)
      966    0.002    0.000    0.004    0.000 base_layer.py:2272(_call_arg_was_passed)
      108    0.000    0.000    0.000    0.000 base_layer.py:2283(_get_call_arg_value)
       35    0.000    0.000    0.005    0.000 base_layer.py:2293(_set_connectivity_metadata_)
       35    0.000    0.000    0.004    0.000 base_layer.py:2317(_add_inbound_node)
       35    0.000    0.000    0.000    0.000 base_layer.py:2329(<lambda>)
       35    0.000    0.000    0.000    0.000 base_layer.py:2331(<lambda>)
       35    0.000    0.000    0.000    0.000 base_layer.py:2333(<lambda>)
        3    0.000    0.000    0.000    0.000 base_layer.py:2358(_get_node_attribute_at_index)
      483    0.001    0.000    1.800    0.004 base_layer.py:2394(_maybe_build)
       70    0.000    0.000    0.000    0.000 base_layer.py:2408(<genexpr>)
       35    0.000    0.000    0.000    0.000 base_layer.py:2409(<lambda>)
     41/3    0.000    0.000    0.001    0.000 base_layer.py:2446(_get_trainable_state)
     1312    0.002    0.000    0.008    0.000 base_layer.py:2469(_obj_reference_counts)
     1557    0.001    0.000    0.002    0.000 base_layer.py:2476(_maybe_create_attribute)
      656    0.002    0.000    0.009    0.000 base_layer.py:2493(__delattr__)
7486/4292    0.011    0.000    1.751    0.000 base_layer.py:2543(__setattr__)
       10    0.000    0.000    0.000    0.000 base_layer.py:2579(<genexpr>)
       85    0.000    0.000    0.000    0.000 base_layer.py:2604(<genexpr>)
       50    0.000    0.000    0.000    0.000 base_layer.py:2608(<genexpr>)
       41    0.000    0.000    0.000    0.000 base_layer.py:2654(_attribute_sentinel)
       41    0.000    0.000    0.004    0.000 base_layer.py:2665(_init_call_fn_args)
       41    0.000    0.000    0.003    0.000 base_layer.py:2677(_call_full_argspec)
       41    0.000    0.000    0.003    0.000 base_layer.py:2684(_call_fn_args)
       38    0.000    0.000    0.000    0.000 base_layer.py:2693(_call_accepts_kwargs)
       38    0.000    0.000    0.000    0.000 base_layer.py:2698(_should_compute_mask)
      519    0.001    0.000    0.001    0.000 base_layer.py:2716(_eager_losses)
       41    0.001    0.000    0.010    0.000 base_layer.py:278(__init__)
       35    0.000    0.000    0.000    0.000 base_layer.py:386(build)
       56    0.001    0.000    0.063    0.001 base_layer.py:440(add_weight)
      174    0.000    0.000    0.000    0.000 base_layer.py:741(compute_mask)
   483/71    0.018    0.000    3.512    0.049 base_layer.py:763(__call__)
      966    0.001    0.000    0.001    0.000 base_layer.py:812(<genexpr>)
      966    0.001    0.000    0.003    0.000 base_layer.py:887(<genexpr>)
      142    0.000    0.000    0.000    0.000 base_layer.py:976(dtype)
     1154    0.000    0.000    0.000    0.000 base_layer.py:981(name)
       38    0.000    0.000    0.000    0.000 base_layer.py:986(dynamic)
       56    0.000    0.000    0.011    0.000 base_layer_utils.py:121(<lambda>)
      483    0.001    0.000    0.005    0.000 base_layer_utils.py:160(have_all_keras_metadata)
      966    0.000    0.000    0.001    0.000 base_layer_utils.py:161(<genexpr>)
      483    0.001    0.000    0.013    0.000 base_layer_utils.py:272(needs_keras_history)
      142    0.000    0.000    0.000    0.000 base_layer_utils.py:294(<genexpr>)
     1069    0.001    0.000    0.008    0.000 base_layer_utils.py:301(is_in_keras_graph)
      986    0.001    0.000    0.043    0.000 base_layer_utils.py:306(is_in_eager_or_tf_function)
      951    0.003    0.000    0.040    0.000 base_layer_utils.py:311(is_in_tf_function)
       36    0.002    0.000    0.011    0.000 base_layer_utils.py:329(uses_keras_history)
       36    0.000    0.000    0.001    0.000 base_layer_utils.py:374(mark_checked)
       36    0.000    0.000    0.000    0.000 base_layer_utils.py:384(_mark_checked)
     2091    0.001    0.000    0.002    0.000 base_layer_utils.py:390(call_context)
      966    0.002    0.000    0.005    0.000 base_layer_utils.py:420(enter)
     1069    0.002    0.000    0.007    0.000 base_layer_utils.py:453(in_keras_graph)
      483    0.001    0.000    0.004    0.000 base_layer_utils.py:471(autocast_context_manager)
      483    0.001    0.000    0.001    0.000 base_layer_utils.py:488(is_subclassed)
       56    0.001    0.000    0.057    0.001 base_layer_utils.py:51(make_variable)
       35    0.000    0.000    0.019    0.001 base_layer_utils.py:627(mark_as_return)
       35    0.000    0.000    0.018    0.001 base_layer_utils.py:630(_mark_as_return)
      117    0.000    0.000    0.000    0.000 base_layer_utils.py:703(v2_dtype_behavior_enabled)
        1    0.000    0.000    0.002    0.002 box.py:24(__init__)
        3    0.000    0.000    0.000    0.000 box.py:39(_get_precision)
        2    0.000    0.000    0.002    0.001 break_statements.py:152(transform)
     3788    0.003    0.000    0.008    0.000 c_api_util.py:157(tf_buffer)
     3144    0.012    0.000    0.012    0.000 c_api_util.py:190(tf_output)
       60    0.000    0.000    0.002    0.000 c_api_util.py:45(__init__)
      544    0.001    0.000    0.229    0.000 c_api_util.py:53(__del__)
       58    0.000    0.000    0.000    0.000 c_api_util.py:86(__init__)
      538    0.002    0.000    0.090    0.000 c_api_util.py:94(__del__)
        2    0.000    0.000    0.009    0.004 call_trees.py:111(visit_FunctionDef)
        2    0.000    0.000    0.008    0.004 call_trees.py:141(visit_With)
        6    0.000    0.000    0.000    0.000 call_trees.py:146(_args_to_tuple)
        6    0.000    0.000    0.001    0.000 call_trees.py:164(_kwargs_to_dict)
    12/11    0.000    0.000    0.007    0.001 call_trees.py:175(visit_Call)
        2    0.000    0.000    0.009    0.004 call_trees.py:226(transform)
        2    0.000    0.000    0.000    0.000 call_trees.py:43(__init__)
        6    0.000    0.000    0.000    0.000 call_trees.py:58(__init__)
        6    0.000    0.000    0.000    0.000 call_trees.py:63(_consume_args)
        5    0.000    0.000    0.000    0.000 call_trees.py:69(add_arg)
        6    0.000    0.000    0.000    0.000 call_trees.py:81(finalize)
        6    0.000    0.000    0.000    0.000 call_trees.py:85(to_ast)
       44    0.000    0.000    0.001    0.000 cfg.py:175(__init__)
       44    0.000    0.000    0.001    0.000 cfg.py:202(reset)
       44    0.000    0.000    0.000    0.000 cfg.py:203(<dictcomp>)
       44    0.000    0.000    0.000    0.000 cfg.py:206(<dictcomp>)
       44    0.001    0.000    0.016    0.000 cfg.py:210(_visit_internal)
       22    0.000    0.000    0.012    0.001 cfg.py:234(visit_forward)
       22    0.000    0.000    0.004    0.000 cfg.py:237(visit_reverse)
       22    0.000    0.000    0.000    0.000 cfg.py:277(__init__)
       44    0.000    0.000    0.000    0.000 cfg.py:281(reset)
      151    0.000    0.000    0.000    0.000 cfg.py:322(_connect_nodes)
      173    0.001    0.000    0.001    0.000 cfg.py:337(_add_new_node)
       16    0.000    0.000    0.000    0.000 cfg.py:361(begin_statement)
       16    0.000    0.000    0.000    0.000 cfg.py:370(end_statement)
      143    0.000    0.000    0.001    0.000 cfg.py:380(add_ordinary_node)
       30    0.000    0.000    0.000    0.000 cfg.py:395(_add_jump_node)
       30    0.000    0.000    0.000    0.000 cfg.py:414(_connect_jump_to_finally_sections)
       30    0.000    0.000    0.000    0.000 cfg.py:427(add_exit_node)
        8    0.000    0.000    0.000    0.000 cfg.py:458(connect_raise_node)
       22    0.000    0.000    0.000    0.000 cfg.py:473(enter_section)
       22    0.000    0.000    0.000    0.000 cfg.py:485(exit_section)
        8    0.000    0.000    0.000    0.000 cfg.py:528(enter_cond_section)
       16    0.000    0.000    0.000    0.000 cfg.py:542(new_cond_branch)
        8    0.000    0.000    0.000    0.000 cfg.py:556(exit_cond_section)
        8    0.000    0.000    0.000    0.000 cfg.py:563(enter_except_section)
       22    0.000    0.000    0.002    0.000 cfg.py:588(build)
       22    0.000    0.000    0.000    0.000 cfg.py:642(__init__)
       30    0.000    0.000    0.000    0.000 cfg.py:651(_enter_lexical_scope)
       30    0.000    0.000    0.000    0.000 cfg.py:654(_exit_lexical_scope)
       30    0.000    0.000    0.000    0.000 cfg.py:658(_get_enclosing_finally_scopes)
        8    0.000    0.000    0.000    0.000 cfg.py:667(_get_enclosing_except_scopes)
      143    0.000    0.000    0.010    0.000 cfg.py:676(_process_basic_statement)
       30    0.000    0.000    0.000    0.000 cfg.py:680(_process_exit_statement)
      173    0.000    0.000    0.000    0.000 cfg.py:73(__init__)
       22    0.000    0.000    0.013    0.001 cfg.py:730(visit_FunctionDef)
       22    0.000    0.000    0.000    0.000 cfg.py:754(visit_Return)
        8    0.000    0.000    0.000    0.000 cfg.py:763(visit_Expr)
       95    0.000    0.000    0.006    0.000 cfg.py:766(visit_Assign)
      173    0.000    0.000    0.001    0.000 cfg.py:78(freeze)
        8    0.000    0.000    0.000    0.000 cfg.py:787(visit_Raise)
        8    0.000    0.000    0.000    0.000 cfg.py:879(visit_ExceptHandler)
        8    0.000    0.000    0.001    0.000 cfg.py:893(visit_Try)
       18    0.000    0.000    0.008    0.000 cfg.py:935(visit_With)
       22    0.000    0.000    0.013    0.001 cfg.py:943(build)
        2    0.000    0.000    0.000    0.000 codecs.py:185(__init__)
    19851    0.010    0.000    0.021    0.000 compat.py:114(as_str)
      246    0.000    0.000    0.000    0.000 compat.py:39(_date_to_date_number)
      246    0.000    0.000    0.000    0.000 compat.py:63(forward_compatible)
     6209    0.007    0.000    0.012    0.000 compat.py:64(as_bytes)
    19851    0.008    0.000    0.011    0.000 compat.py:90(as_text)
        4    0.000    0.000    0.000    0.000 compilerop.py:137(check_linecache_ipython)
       20    0.000    0.000    0.004    0.000 cond_v2.py:1127(_set_read_only_resource_inputs_attr)
       20    0.000    0.000    0.000    0.000 cond_v2.py:1147(<listcomp>)
       20    0.001    0.000    0.076    0.004 cond_v2.py:191(_build_cond)
       20    0.000    0.000    0.000    0.000 cond_v2.py:257(<listcomp>)
       20    0.000    0.000    0.000    0.000 cond_v2.py:260(<listcomp>)
       20    0.000    0.000    0.000    0.000 cond_v2.py:269(<listcomp>)
       20    0.000    0.000    0.019    0.001 cond_v2.py:295(<listcomp>)
       20    0.000    0.000    0.001    0.000 cond_v2.py:496(_make_inputs_match)
       40    0.000    0.000    0.000    0.000 cond_v2.py:524(<listcomp>)
       20    0.001    0.000    0.310    0.015 cond_v2.py:58(cond_v2)
       20    0.000    0.000    0.001    0.000 cond_v2.py:619(_make_indexed_slices_indices_types_match)
       20    0.000    0.000    0.000    0.000 cond_v2.py:629(<listcomp>)
       20    0.000    0.000    0.000    0.000 cond_v2.py:632(<listcomp>)
      120    0.000    0.000    0.000    0.000 cond_v2.py:637(<genexpr>)
       20    0.000    0.000    0.000    0.000 cond_v2.py:683(_get_op_and_outputs)
       20    0.000    0.000    0.001    0.000 cond_v2.py:692(_pack_sequence_as)
       20    0.000    0.000    0.000    0.000 cond_v2.py:760(_check_same_outputs)
       20    0.000    0.000    0.001    0.000 cond_v2.py:804(_get_output_shapes)
       20    0.000    0.000    0.000    0.000 cond_v2.py:814(verify_captures)
       20    0.000    0.000    0.000    0.000 cond_v2.py:820(<dictcomp>)
        2    0.000    0.000    0.003    0.002 conditional_expressions.py:44(transform)
      805    0.000    0.000    0.001    0.000 config_lib.py:33(matches)
      768    0.000    0.000    0.001    0.000 config_lib.py:50(get_action)
       37    0.000    0.000    0.000    0.000 config_lib.py:62(get_action)
   615293    0.545    0.000    7.645    0.000 constant_op.py:164(constant)
   615293    0.953    0.000    7.101    0.000 constant_op.py:265(_constant_impl)
   134388    0.070    0.000    1.363    0.000 constant_op.py:318(_constant_tensor_conversion_function)
       43    0.000    0.000    0.001    0.000 constant_op.py:330(_tensor_shape_tensor_conversion_function)
   615191    4.950    0.000    5.772    0.000 constant_op.py:68(convert_to_eager_tensor)
      122    0.000    0.000    0.000    0.000 constraints.py:280(get)
    15014    0.010    0.000    0.010    0.000 context.py:1021(function_call_options)
       58    0.000    0.000    0.005    0.000 context.py:1050(add_function)
      538    0.002    0.000    0.543    0.001 context.py:1096(remove_function)
     2382    0.002    0.000    0.002    0.000 context.py:1146(op_callbacks)
    15014    0.004    0.000    0.004    0.000 context.py:127(executor_type)
    29988    0.008    0.000    0.008    0.000 context.py:135(config_proto_serialized)
        2    0.000    0.000    0.000    0.000 context.py:1562(disable_run_metadata)
        1    0.000    0.000    2.249    2.249 context.py:1583(export_run_metadata)
     9026    0.002    0.000    0.002    0.000 context.py:1601(context_switches)
        1    0.000    0.000    0.000    0.000 context.py:1616(__init__)
        1    0.000    0.000    0.000    0.000 context.py:1621(__enter__)
        1    0.000    0.000    0.000    0.000 context.py:1655(__exit__)
  2599334    0.638    0.000    0.638    0.000 context.py:1702(context)
   123774    0.034    0.000    0.034    0.000 context.py:1709(context_safe)
    15053    0.026    0.000    0.043    0.000 context.py:1714(ensure_initialized)
       33    0.000    0.000    0.000    0.000 context.py:1724(global_seed)
   123658    0.196    0.000    0.334    0.000 context.py:1734(executing_eagerly)
       56    0.000    0.000    0.000    0.000 context.py:1864(shared_name)
     2119    0.002    0.000    0.006    0.000 context.py:1885(graph_mode)
     1596    0.002    0.000    0.005    0.000 context.py:1890(eager_mode)
        1    0.000    0.000    0.000    0.000 context.py:1900(device)
     3715    0.005    0.000    0.010    0.000 context.py:218(push)
       58    0.000    0.000    0.005    0.000 context.py:2230(add_function)
      538    0.001    0.000    0.545    0.001 context.py:2235(remove_function)
      116    0.000    0.000    0.000    0.000 context.py:2248(_tmp_in_graph_mode)
     3715    0.003    0.000    0.004    0.000 context.py:238(pop)
   646128    0.286    0.000    0.286    0.000 context.py:492(ensure_initialized)
   631075    0.213    0.000    0.213    0.000 context.py:723(_handle)
     7430    0.008    0.000    0.013    0.000 context.py:747(_mode)
  2710312    1.143    0.000    1.143    0.000 context.py:765(executing_eagerly)
    44845    0.014    0.000    0.014    0.000 context.py:777(scope_name)
    89688    0.047    0.000    0.047    0.000 context.py:782(scope_name)
   630480    0.216    0.000    0.216    0.000 context.py:787(device_name)
        2    0.000    0.000    0.000    0.000 context.py:792(device_spec)
        2    0.000    0.000    0.000    0.000 context.py:797(_set_device)
        1    0.000    0.000    0.000    0.000 context.py:801(device)
     2119    0.005    0.000    0.022    0.000 contextlib.py:129(contextmanager)
    18838    0.012    0.000    0.043    0.000 contextlib.py:157(helper)
        3    0.000    0.000    0.000    0.000 contextlib.py:36(_recreate_cm)
        3    0.000    0.000    0.000    0.000 contextlib.py:49(inner)
    18838    0.026    0.000    0.031    0.000 contextlib.py:59(__init__)
18838/11274    0.008    0.000    0.159    0.000 contextlib.py:79(__enter__)
18838/11297    0.017    0.000    0.075    0.000 contextlib.py:85(__exit__)
        2    0.000    0.000    0.001    0.001 continue_statements.py:117(_visit_non_loop_body)
        2    0.000    0.000    0.002    0.001 continue_statements.py:143(visit_With)
        2    0.000    0.000    0.002    0.001 continue_statements.py:161(transform)
        2    0.000    0.000    0.000    0.000 continue_statements.py:29(__init__)
        4    0.000    0.000    0.000    0.000 continue_statements.py:52(__init__)
        7    0.000    0.000    0.000    0.000 continue_statements.py:77(_postprocess_statement)
        2    0.000    0.000    0.003    0.002 control_flow.py:49(visit_FunctionDef)
        2    0.000    0.000    0.003    0.002 control_flow.py:528(transform)
       20    0.000    0.000    0.310    0.015 control_flow_ops.py:1098(cond)
     2423    0.001    0.000    0.002    0.000 control_flow_util.py:180(GetOutputContext)
     2481    0.001    0.000    0.001    0.000 control_flow_util.py:191(GetContainingWhileContext)
       80    0.000    0.000    0.000    0.000 control_flow_util.py:213(GetContainingXLAContext)
     2423    0.003    0.000    0.005    0.000 control_flow_util.py:266(CheckInputFromValidContext)
       20    0.000    0.000    0.000    0.000 control_flow_util.py:50(EnableControlFlowV2)
       80    0.000    0.000    0.000    0.000 control_flow_util.py:68(InXlaContext)
       40    0.000    0.000    0.000    0.000 control_flow_util.py:73(GraphOrParentsInXlaContext)
     2481    0.002    0.000    0.004    0.000 control_flow_util.py:82(IsInWhileLoop)
       20    0.000    0.000    0.000    0.000 control_flow_util_v2.py:113(maybe_propagate_compile_time_consts_in_xla)
       20    0.000    0.000    0.000    0.000 control_flow_util_v2.py:231(output_all_intermediates)
       20    0.000    0.000    0.000    0.000 control_flow_util_v2.py:42(in_defun)
       40    0.000    0.000    0.017    0.000 control_flow_util_v2.py:53(create_new_tf_function)
       40    0.000    0.000    0.000    0.000 control_flow_util_v2.py:68(unique_fn_name)
       20    0.000    0.000    0.001    0.000 control_flow_util_v2.py:85(maybe_set_lowering_attr)
       12    0.000    0.000    0.000    0.000 conv_utils.py:189(normalize_data_format)
       10    0.000    0.000    0.000    0.000 conv_utils.py:200(normalize_padding)
       10    0.000    0.000    0.000    0.000 conv_utils.py:28(convert_data_format)
       30    0.000    0.000    0.000    0.000 conv_utils.py:51(normalize_tuple)
      123    0.001    0.000    0.002    0.000 conversion.py:119(has)
       38    0.000    0.000    0.000    0.000 conversion.py:125(__getitem__)
       70    0.000    0.000    0.000    0.000 conversion.py:147(_get_key)
       91    0.000    0.000    0.000    0.000 conversion.py:163(_get_key)
        2    0.000    0.000    0.012    0.006 conversion.py:182(_wrap_into_dynamic_factory)
       35    0.000    0.000    0.264    0.008 conversion.py:254(_convert_with_cache)
       35    0.001    0.000    0.001    0.000 conversion.py:298(_instantiate)
       35    0.000    0.000    0.000    0.000 conversion.py:316(<genexpr>)
       35    0.000    0.000    0.266    0.008 conversion.py:340(convert)
    54/20    0.001    0.000    0.004    0.000 conversion.py:366(is_whitelisted)
       88    0.000    0.000    0.001    0.000 conversion.py:471(is_in_whitelist_cache)
        3    0.000    0.000    0.000    0.000 conversion.py:479(cache_whitelisted)
        2    0.000    0.000    0.231    0.115 conversion.py:488(convert_entity_to_ast)
        2    0.000    0.000    0.000    0.000 conversion.py:635(_add_reserved_symbol)
        2    0.000    0.000    0.000    0.000 conversion.py:646(_add_self_references)
        2    0.000    0.000    0.231    0.115 conversion.py:669(convert_func_to_ast)
        2    0.000    0.000    0.216    0.108 conversion.py:729(node_to_graph)
       35    0.000    0.000    0.000    0.000 conversion.py:90(get_module)
       35    0.000    0.000    0.000    0.000 conversion.py:93(get_factory)
       71    0.000    0.000    0.000    0.000 converter.py:158(__init__)
      357    0.000    0.000    0.000    0.000 converter.py:175(as_tuple)
      121    0.000    0.000    0.000    0.000 converter.py:179(__hash__)
      118    0.000    0.000    0.000    0.000 converter.py:182(__eq__)
       74    0.000    0.000    0.000    0.000 converter.py:189(uses)
       35    0.000    0.000    0.000    0.000 converter.py:193(call_options)
        2    0.000    0.000    0.002    0.001 converter.py:201(to_ast)
        2    0.000    0.000    0.000    0.000 converter.py:221(list_of_features)
        2    0.000    0.000    0.000    0.000 converter.py:223(<genexpr>)
        2    0.000    0.000    0.000    0.000 converter.py:268(__init__)
       22    0.000    0.000    0.000    0.000 converter.py:282(__init__)
  2124/22    0.002    0.000    0.049    0.002 converter.py:337(visit)
      201    0.000    0.000    0.000    0.000 converter.py:352(__init__)
       22    0.000    0.000    0.166    0.008 converter.py:357(standard_analysis)
       20    0.000    0.000    0.205    0.010 converter.py:387(apply_)
       10    0.000    0.000    0.009    0.001 convolutional.py:102(__init__)
       10    0.000    0.000    0.025    0.003 convolutional.py:151(build)
      118    0.001    0.000    0.213    0.002 convolutional.py:193(call)
       20    0.000    0.000    0.000    0.000 convolutional.py:284(_get_channel_axis)
       10    0.000    0.000    0.000    0.000 convolutional.py:290(_get_input_channel)
       10    0.000    0.000    0.000    0.000 convolutional.py:297(_get_padding_op)
      118    0.001    0.000    0.003    0.000 convolutional.py:306(_recreate_conv_op)
       10    0.000    0.000    0.010    0.001 convolutional.py:565(__init__)
        3    0.000    0.000    0.002    0.001 core.py:1116(__init__)
        3    0.000    0.000    0.007    0.002 core.py:1147(build)
       56    0.001    0.000    0.178    0.003 core.py:1179(call)
        1    0.000    0.000    0.000    0.000 core.py:132(unwrapped)
        1    0.000    0.000    0.000    0.000 core.py:208(__init__)
        2    0.000    0.000    0.001    0.000 core.py:629(__init__)
       37    0.001    0.000    0.046    0.001 core.py:634(call)
       37    0.000    0.000    0.002    0.000 core.py:669(compute_output_shape)
      296    0.016    0.000    0.016    0.000 ctypeslib.py:350(_ctype_ndarray)
      296    0.002    0.000    0.003    0.000 ctypeslib.py:376(_ctype_from_dtype_scalar)
      296    0.001    0.000    0.004    0.000 ctypeslib.py:455(_ctype_from_dtype)
      296    0.001    0.000    0.005    0.000 ctypeslib.py:464(as_ctypes_type)
      296    0.005    0.000    0.026    0.000 ctypeslib.py:526(as_ctypes)
      802    0.011    0.000    0.039    0.000 custom_gradient.py:44(copy_handle_data)
      782    0.003    0.000    0.003    0.000 custom_gradient.py:74(<listcomp>)
      782    0.001    0.000    0.001    0.000 custom_gradient.py:76(<listcomp>)
      782    0.001    0.000    0.002    0.000 custom_gradient.py:77(<listcomp>)
  696/676    0.001    0.000    0.004    0.000 data_structures.py:103(sticky_attribute_assignment)
       20    0.000    0.000    0.000    0.000 data_structures.py:140(__init__)
       10    0.000    0.000    0.000    0.000 data_structures.py:152(__init__)
       30    0.000    0.000    0.000    0.000 data_structures.py:161(_attribute_sentinel)
       20    0.000    0.000    0.000    0.000 data_structures.py:173(_track_value)
      118    0.000    0.000    0.001    0.000 data_structures.py:195(_layers)
      118    0.000    0.000    0.002    0.000 data_structures.py:208(layers)
       10    0.000    0.000    0.000    0.000 data_structures.py:309(__init__)
       20    0.000    0.000    0.000    0.000 data_structures.py:330(_name_element)
      118    0.000    0.000    0.000    0.000 data_structures.py:333(_values)
      336    0.000    0.000    0.000    0.000 data_structures.py:377(__getitem__)
       10    0.000    0.000    0.000    0.000 data_structures.py:383(__len__)
       10    0.000    0.000    0.000    0.000 data_structures.py:414(__init__)
       20    0.000    0.000    0.000    0.000 data_structures.py:431(_non_append_mutation)
       20    0.000    0.000    0.000    0.000 data_structures.py:447(_external_modification)
       10    0.000    0.000    0.000    0.000 data_structures.py:475(_make_storage)
       10    0.000    0.000    0.000    0.000 data_structures.py:479(_check_external_modification)
       10    0.000    0.000    0.000    0.000 data_structures.py:487(_update_snapshot)
       10    0.000    0.000    0.000    0.000 data_structures.py:525(__setitem__)
       20    0.000    0.000    0.000    0.000 data_structures.py:576(__eq__)
       20    0.000    0.000    0.000    0.000 data_structures.py:614(_track_value)
       33    0.000    0.000    0.000    0.000 data_structures.py:66(_should_wrap_tuple)
  768/676    0.001    0.000    0.002    0.000 data_structures.py:80(wrap_or_unwrap)
     2481    0.002    0.000    0.007    0.000 dataset_ops.py:4530(_resource_resolver)
        2    0.000    0.000    0.000    0.000 def_function.py:318(__init__)
        6    0.000    0.000    0.000    0.000 def_function.py:321(__del__)
        2    0.000    0.000    0.000    0.000 def_function.py:338(__init__)
        4    0.000    0.000    0.001    0.000 def_function.py:420(_defun_with_scope)
       18    0.000    0.000    1.562    0.087 def_function.py:424(wrapped_fn)
        4    0.000    0.000    0.001    0.000 def_function.py:448(_defun)
        2    0.000    0.000    0.437    0.218 def_function.py:472(_initialize)
        2    0.000    0.000    0.000    0.000 def_function.py:53(__init__)
    29988    0.045    0.000    0.045    0.000 def_function.py:555(_get_tracing_count)
    14994    0.099    0.000   17.339    0.001 def_function.py:560(__call__)
       18    0.000    0.000    0.000    0.000 def_function.py:58(called_with_tracing)
    14994    0.075    0.000   17.126    0.001 def_function.py:602(_call)
    14976    0.026    0.000    0.026    0.000 def_function.py:68(called_without_tracing)
        8    0.000    0.000    0.000    0.000 def_function.py:710(python_function)
        6    0.000    0.000    0.000    0.000 def_function.py:715(input_signature)
       18    0.000    0.000    0.000    0.000 def_function.py:76(get_tracing_count)
    14994    0.058    0.000    0.121    0.000 def_function.py:963(__get__)
   116/96    0.001    0.000    0.323    0.003 deprecation.py:473(new_func)
      138    0.000    0.000    0.000    0.000 deprecation.py:583(deprecated_argument_lookup)
        1    0.000    0.000    0.000    0.000 device.py:45(is_device_spec)
      118    0.000    0.000    0.000    0.000 device_context.py:21(enclosing_tpu_context)
       41    0.000    0.000    0.001    0.000 directives.py:117(visit_Name)
       14    0.000    0.000    0.001    0.000 directives.py:126(visit_Attribute)
        5    0.000    0.000    0.001    0.000 directives.py:134(visit_Assign)
        2    0.000    0.000    0.002    0.001 directives.py:177(transform)
        2    0.000    0.000    0.000    0.000 directives.py:50(__init__)
        1    0.000    0.000    0.000    0.000 discrete.py:13(__init__)
134726/134706    0.081    0.000    3.013    0.000 dispatch.py:177(wrapper)
      486    0.000    0.000    0.000    0.000 distribute_lib.py:2677(value_container)
      486    0.000    0.000    0.000    0.000 distribute_lib.py:627(extended)
      144    0.000    0.000    0.001    0.000 distribution_strategy_context.py:179(get_strategy)
      141    0.000    0.000    0.001    0.000 distribution_strategy_context.py:199(has_strategy)
      486    0.001    0.000    0.003    0.000 distribution_strategy_context.py:215(get_strategy_and_replica_context)
      141    0.000    0.000    0.000    0.000 distribution_strategy_context.py:312(_get_default_strategy)
      630    0.000    0.000    0.000    0.000 distribution_strategy_context.py:337(_get_default_replica_mode)
      630    0.002    0.000    0.004    0.000 distribution_strategy_context.py:79(_get_per_thread_mode)
       74    0.000    0.000    0.000    0.000 dtypes.py:103(as_numpy_dtype)
   309793    0.664    0.000    1.100    0.000 dtypes.py:172(is_compatible_with)
   941272    1.326    0.000    1.329    0.000 dtypes.py:192(__eq__)
   452980    0.282    0.000    0.842    0.000 dtypes.py:205(__ne__)
   805476    0.365    0.000    0.482    0.000 dtypes.py:606(as_dtype)
  1699685    1.610    0.000    1.610    0.000 dtypes.py:71(_is_ref_dtype)
  1699685    0.844    0.000    2.454    0.000 dtypes.py:84(base_dtype)
      597    0.001    0.000    0.001    0.000 enum.py:267(__call__)
        2    0.000    0.000    0.000    0.000 enum.py:297(__contains__)
      597    0.000    0.000    0.000    0.000 enum.py:517(__new__)
    25780    0.007    0.000    0.010    0.000 enum.py:581(__hash__)
     1719    0.001    0.000    0.002    0.000 errors_impl.py:117(__str__)
     1719    0.002    0.000    0.005    0.000 errors_impl.py:264(__init__)
     1719    0.003    0.000    0.003    0.000 errors_impl.py:64(__init__)
     1719    0.000    0.000    0.000    0.000 errors_impl.py:86(message)
      256    0.000    0.000    0.001    0.000 execute.py:159(make_float)
      944    0.001    0.000    0.001    0.000 execute.py:166(make_int)
      420    0.000    0.000    0.001    0.000 execute.py:177(make_str)
      359    0.000    0.000    0.000    0.000 execute.py:184(make_bool)
      786    0.001    0.000    0.003    0.000 execute.py:191(make_type)
       53    0.000    0.000    0.000    0.000 execute.py:201(make_shape)
      588    0.002    0.000    0.019    0.000 execute.py:236(args_to_matching_eager)
      294    0.000    0.000    0.001    0.000 execute.py:271(<listcomp>)
    15288    0.055    0.000   12.284    0.001 execute.py:33(quick_execute)
        1    0.000    0.000    0.000    0.000 ezpickle.py:20(__init__)
      338    0.000    0.000    0.000    0.000 fromnumeric.py:2232(_any_dispatcher)
      338    0.001    0.000    0.006    0.000 fromnumeric.py:2236(any)
       47    0.000    0.000    0.000    0.000 fromnumeric.py:2320(_all_dispatcher)
       47    0.000    0.000    0.001    0.000 fromnumeric.py:2324(all)
      294    0.000    0.000    0.000    0.000 fromnumeric.py:2546(_amax_dispatcher)
      294    0.002    0.000    0.008    0.000 fromnumeric.py:2551(amax)
      370    0.000    0.000    0.000    0.000 fromnumeric.py:2838(_prod_dispatcher)
      370    0.001    0.000    0.007    0.000 fromnumeric.py:2843(prod)
     1049    0.005    0.000    0.019    0.000 fromnumeric.py:73(_wrapreduction)
     1049    0.001    0.000    0.001    0.000 fromnumeric.py:74(<dictcomp>)
       58    0.000    0.000    0.001    0.000 func_graph.py:1011(<listcomp>)
      209    0.000    0.000    0.000    0.000 func_graph.py:1018(<genexpr>)
     2063    0.003    0.000    0.008    0.000 func_graph.py:1048(device_stack_has_callable)
     2063    0.001    0.000    0.002    0.000 func_graph.py:1050(<genexpr>)
      116    0.000    0.000    0.001    0.000 func_graph.py:1054(check_mutation)
       58    0.000    0.000    0.000    0.000 func_graph.py:1074(flatten)
       58    0.000    0.000    0.000    0.000 func_graph.py:1086(<listcomp>)
       20    0.000    0.000    0.001    0.000 func_graph.py:1091(pack_sequence_as)
      802    0.005    0.000    0.310    0.000 func_graph.py:1116(_create_substitute_placeholder)
       58    0.000    0.000    0.014    0.000 func_graph.py:1129(_get_defun_inputs_from_args)
       35    0.000    0.000    0.000    0.000 func_graph.py:1135(_get_composite_tensor_spec)
      116    0.001    0.000    0.016    0.000 func_graph.py:1141(_get_defun_inputs)
       35    0.000    0.000    0.000    0.000 func_graph.py:1192(<listcomp>)
       35    0.000    0.000    0.000    0.000 func_graph.py:1194(<listcomp>)
       58    0.000    0.000    0.001    0.000 func_graph.py:1258(_get_defun_inputs_from_kwargs)
      544    0.006    0.000    0.560    0.001 func_graph.py:1269(dismantle_func_graph)
       18    0.000    0.000    0.000    0.000 func_graph.py:127(<listcomp>)
      116    0.000    0.000    0.002    0.000 func_graph.py:130(<listcomp>)
       60    0.001    0.000    0.008    0.000 func_graph.py:165(__init__)
      742    0.002    0.000    0.009    0.000 func_graph.py:265(watch_variable)
      822    0.001    0.000    0.003    0.000 func_graph.py:321(control_dependencies)
     2119    0.006    0.000    0.060    0.000 func_graph.py:362(as_default)
     4238    0.025    0.000    0.123    0.000 func_graph.py:365(inner_cm)
     1368    0.001    0.000    0.001    0.000 func_graph.py:429(outer_graph)
       58    0.000    0.000    0.000    0.000 func_graph.py:493(variables)
       58    0.000    0.000    0.000    0.000 func_graph.py:495(<listcomp>)
2484/2464    0.019    0.000    0.717    0.000 func_graph.py:530(_create_op_internal)
     3356    0.006    0.000    0.330    0.000 func_graph.py:597(capture)
      802    0.004    0.000    0.319    0.000 func_graph.py:646(_capture_helper)
      802    0.002    0.000    0.002    0.000 func_graph.py:664(add_capture)
       40    0.000    0.000    0.000    0.000 func_graph.py:678(reset_captures)
      116    0.000    0.000    0.007    0.000 func_graph.py:69(convert_structure_to_signature)
      544    0.007    0.000    0.016    0.000 func_graph.py:692(clear_captures)
       98    0.000    0.000    0.000    0.000 func_graph.py:730(external_captures)
       98    0.000    0.000    0.000    0.000 func_graph.py:733(<listcomp>)
       58    0.000    0.000    0.000    0.000 func_graph.py:735(internal_captures)
       58    0.000    0.000    0.000    0.000 func_graph.py:738(<listcomp>)
       18    0.000    0.000    0.000    0.000 func_graph.py:740(deferred_external_captures)
       18    0.000    0.000    0.000    0.000 func_graph.py:743(<listcomp>)
       58    0.000    0.000    0.000    0.000 func_graph.py:745(deferred_internal_captures)
       58    0.000    0.000    0.000    0.000 func_graph.py:748(<listcomp>)
       58    0.003    0.000    2.048    0.035 func_graph.py:796(func_graph_from_py_func)
       35    0.000    0.000    0.002    0.000 func_graph.py:82(encode_arg)
       58    0.000    0.000    0.000    0.000 func_graph.py:914(<listcomp>)
      151    0.001    0.000    0.084    0.001 func_graph.py:925(convert)
       18    0.000    0.000    1.561    0.087 func_graph.py:953(wrapper)
       36    0.000    0.000    0.000    0.000 func_graph.py:97(<genexpr>)
44688/29988    0.027    0.000    0.123    0.000 function.py:105(<lambda>)
       18    0.000    0.000    0.029    0.002 function.py:1498(__init__)
       89    0.000    0.000    0.002    0.000 function.py:1551(<genexpr>)
    14994    0.073    0.000   14.068    0.001 function.py:1647(_filtered_call)
    29988    0.033    0.000    0.047    0.000 function.py:1662(<genexpr>)
    14994    0.195    0.000   13.851    0.001 function.py:1667(_call_flat)
    14994    0.049    0.000    0.073    0.000 function.py:1811(captured_inputs)
    14994    0.004    0.000    0.004    0.000 function.py:1817(<listcomp>)
    14994    0.132    0.000    0.879    0.000 function.py:1954(_build_call_outputs)
        6    0.000    0.000    0.001    0.000 function.py:1998(from_function_and_signature)
        6    0.000    0.000    0.000    0.000 function.py:2081(__init__)
        6    0.000    0.000    0.000    0.000 function.py:2100(<dictcomp>)
        6    0.000    0.000    0.000    0.000 function.py:2106(<dictcomp>)
    30032    0.011    0.000    0.011    0.000 function.py:2142(input_signature)
    14996    0.054    0.000    0.257    0.000 function.py:2155(canonicalize_function_inputs)
    14996    0.069    0.000    0.169    0.000 function.py:2242(_convert_numpy_inputs)
        4    0.000    0.000    0.000    0.000 function.py:2320(__init__)
        4    0.000    0.000    0.001    0.000 function.py:2358(__init__)
    14992    0.062    0.000   16.544    0.001 function.py:2416(__call__)
    30026    0.022    0.000    0.033    0.000 function.py:2431(input_signature)
        2    0.000    0.000    0.436    0.218 function.py:2441(_get_concrete_function_internal_garbage_collected)
    14994    0.143    0.000    0.621    0.000 function.py:2570(_cache_key)
       18    0.000    0.000    1.862    0.103 function.py:2639(_create_graph_function)
       18    0.000    0.000    0.000    0.000 function.py:2653(<listcomp>)
    14994    0.081    0.000    2.904    0.000 function.py:2711(_maybe_define_function)
       18    0.000    0.000    0.000    0.000 function.py:277(_parse_func_attrs)
    14994    0.009    0.000    0.009    0.000 function.py:313(__init__)
        4    0.000    0.000    0.001    0.000 function.py:3159(defun_with_attributes)
    14994    0.003    0.000    0.003    0.000 function.py:316(__enter__)
    14994    0.011    0.000    0.011    0.000 function.py:319(__exit__)
        4    0.000    0.000    0.001    0.000 function.py:3196(decorated)
        2    0.000    0.000    0.000    0.000 function.py:3240(__init__)
       18    0.000    0.000    0.000    0.000 function.py:3244(target)
        2    0.000    0.000    0.000    0.000 function.py:3264(class_method_to_instance_method)
       18    0.000    0.000    1.561    0.087 function.py:3282(bound_method_wrapper)
       12    0.000    0.000    0.000    0.000 function.py:3324(__init__)
       36    0.145    0.004    1.365    0.038 function.py:3327(__del__)
       18    0.000    0.000    0.000    0.000 function.py:3341(__init__)
      538    0.002    0.000    0.562    0.001 function.py:3348(__del__)
       18    0.000    0.000    0.000    0.000 function.py:404(_inference_name)
       58    0.000    0.000    0.000    0.000 function.py:429(__init__)
      538    0.001    0.000    0.546    0.001 function.py:432(__del__)
       58    0.003    0.000    0.040    0.001 function.py:460(__init__)
      822    0.000    0.000    0.000    0.000 function.py:470(<genexpr>)
       58    0.000    0.000    0.000    0.000 function.py:471(<listcomp>)
       58    0.000    0.000    0.000    0.000 function.py:489(<listcomp>)
       58    0.000    0.000    0.000    0.000 function.py:490(<listcomp>)
       58    0.000    0.000    0.000    0.000 function.py:491(<listcomp>)
       58    0.000    0.000    0.000    0.000 function.py:493(<listcomp>)
       58    0.000    0.000    0.000    0.000 function.py:522(<listcomp>)
       58    0.000    0.000    0.001    0.000 function.py:523(<listcomp>)
      784    0.000    0.000    0.000    0.000 function.py:532(<genexpr>)
       40    0.000    0.000    0.002    0.000 function.py:537(add_to_graph)
      138    0.000    0.000    0.000    0.000 function.py:549(name)
    14994    0.260    0.000   12.551    0.001 function.py:557(call)
       18    0.000    0.000    0.026    0.001 function.py:645(__init__)
       18    0.000    0.000    0.000    0.000 function.py:807(forward)
59682/14994    0.109    0.000    0.184    0.000 function.py:87(_make_input_signature_hashable)
        2    0.000    0.000    0.011    0.005 function_scopes.py:131(transform)
        4    0.000    0.000    0.000    0.000 function_scopes.py:31(__init__)
        2    0.000    0.000    0.002    0.001 function_scopes.py:38(visit_Return)
        2    0.000    0.000    0.000    0.000 function_scopes.py:46(_function_scope_options)
        2    0.000    0.000    0.011    0.005 function_scopes.py:90(visit_FunctionDef)
       35    0.000    0.000    0.001    0.000 function_wrappers.py:43(__init__)
       35    0.000    0.000    0.000    0.000 function_wrappers.py:70(__enter__)
       35    0.000    0.000    0.000    0.000 function_wrappers.py:79(__exit__)
       35    0.000    0.000    0.000    0.000 function_wrappers.py:87(mark_return_value)
     2121    0.008    0.000    0.015    0.000 functools.py:44(update_wrapper)
     2121    0.002    0.000    0.002    0.000 functools.py:74(wraps)
     2015    0.004    0.000    0.005    0.000 gast.py:15(create_node)
       35    0.000    0.000    0.012    0.000 gast.py:297(parse)
      125    0.000    0.000    0.000    0.000 gast.py:321(copy_location)
      294    0.001    0.000    0.010    0.000 gen_array_ops.py:2211(expand_dims)
       43    0.000    0.000    0.002    0.000 gen_array_ops.py:3265(fill)
      226    0.001    0.000    0.105    0.000 gen_array_ops.py:3873(identity)
      303    0.003    0.000    0.142    0.000 gen_array_ops.py:6383(pad)
      294    0.003    0.000    0.082    0.000 gen_array_ops.py:6449(pad_eager_fallback)
        8    0.000    0.000    0.013    0.002 gen_array_ops.py:6463(pad_v2)
        3    0.000    0.000    0.003    0.001 gen_array_ops.py:6637(placeholder)
       10    0.000    0.000    0.010    0.001 gen_array_ops.py:6771(placeholder_with_default)
       37    0.000    0.000    0.021    0.001 gen_array_ops.py:7997(reshape)
       20    0.000    0.000    0.011    0.001 gen_array_ops.py:9817(squeeze)
       20    0.000    0.000    0.000    0.000 gen_array_ops.py:9873(<listcomp>)
    44688    0.086    0.000    1.483    0.000 gen_array_ops.py:9998(strided_slice)
       10    0.000    0.000    0.024    0.002 gen_functional_ops.py:272(_if)
       10    0.000    0.000    0.000    0.000 gen_functional_ops.py:321(<listcomp>)
       10    0.000    0.000    0.000    0.000 gen_functional_ops.py:328(<listcomp>)
       10    0.000    0.000    0.007    0.001 gen_functional_ops.py:646(stateless_if)
       10    0.000    0.000    0.000    0.000 gen_functional_ops.py:699(<listcomp>)
       10    0.000    0.000    0.000    0.000 gen_functional_ops.py:706(<listcomp>)
       56    0.000    0.000    0.009    0.000 gen_logging_ops.py:23(_assert)
   204785    0.370    0.000    6.447    0.000 gen_math_ops.py:10071(sub)
        1    0.000    0.000    0.001    0.001 gen_math_ops.py:1944(cast)
       13    0.000    0.000    0.000    0.000 gen_math_ops.py:306(add)
   109860    0.208    0.000    4.469    0.000 gen_math_ops.py:3175(equal)
   255856    0.551    0.000    7.082    0.000 gen_math_ops.py:3911(greater)
   348028    0.436    0.000   13.686    0.000 gen_math_ops.py:451(add_v2)
    99646    0.355    0.000    4.657    0.000 gen_math_ops.py:4830(less)
       10    0.000    0.000    0.009    0.001 gen_math_ops.py:5330(logical_and)
       56    0.000    0.000    0.011    0.000 gen_math_ops.py:5389(logical_not)
       56    0.001    0.000    0.088    0.002 gen_math_ops.py:5537(mat_mul)
   174027    0.259    0.000    5.247    0.000 gen_math_ops.py:6060(mul)
   276694    0.401    0.000    7.796    0.000 gen_math_ops.py:7225(real_div)
      118    0.001    0.000    0.059    0.000 gen_nn_ops.py:10328(relu)
      128    0.002    0.000    0.361    0.003 gen_nn_ops.py:4200(fused_batch_norm_v3)
       56    0.000    0.000    0.031    0.001 gen_nn_ops.py:642(bias_add)
      118    0.002    0.000    0.207    0.002 gen_nn_ops.py:856(conv2d)
      118    0.000    0.000    0.001    0.000 gen_nn_ops.py:943(<listcomp>)
      118    0.000    0.000    0.000    0.000 gen_nn_ops.py:954(<listcomp>)
      118    0.000    0.000    0.000    0.000 gen_nn_ops.py:964(<listcomp>)
       13    0.000    0.000    0.001    0.000 gen_random_ops.py:687(random_uniform)
       56    0.000    0.000    0.002    0.000 gen_resource_variable_ops.py:1147(var_handle_op)
       56    0.000    0.000    0.002    0.000 gen_resource_variable_ops.py:121(assign_variable_op)
       56    0.000    0.000    0.001    0.000 gen_resource_variable_ops.py:1229(var_is_initialized_op)
      742    0.006    0.000    0.665    0.001 gen_resource_variable_ops.py:445(read_variable_op)
       66    0.000    0.000    0.001    0.000 generic_utils.py:350(deserialize_keras_object)
       20    0.000    0.000    0.000    0.000 generic_utils.py:742(to_list)
       38    0.000    0.000    0.001    0.000 generic_utils.py:759(to_snake_case)
      483    0.000    0.000    0.001    0.000 generic_utils.py:769(is_all_none)
       47    0.000    0.000    0.000    0.000 generic_utils.py:786(validate_kwargs)
        1    0.000    0.000    0.000    0.000 genericpath.py:117(_splitext)
       42    0.000    0.000    0.001    0.000 genericpath.py:16(exists)
      588    0.001    0.000    0.002    0.000 getlimits.py:365(__new__)
       37    0.000    0.000    0.000    0.000 getlimits.py:497(__init__)
       37    0.000    0.000    0.000    0.000 getlimits.py:521(max)
      820    0.013    0.000    0.257    0.000 graph_only_ops.py:29(graph_placeholder)
        3    0.000    0.000    0.000    0.000 graph_view.py:143(__init__)
       13    0.000    0.000    0.000    0.000 init_ops_v2.py:1021(_assert_float_dtype)
       13    0.000    0.000    0.000    0.000 init_ops_v2.py:1044(__init__)
       13    0.000    0.000    0.004    0.000 init_ops_v2.py:1061(random_uniform)
       23    0.000    0.000    0.003    0.000 init_ops_v2.py:117(__call__)
       20    0.000    0.000    0.003    0.000 init_ops_v2.py:157(__call__)
       13    0.000    0.000    0.000    0.000 init_ops_v2.py:503(__init__)
       13    0.000    0.000    0.004    0.000 init_ops_v2.py:525(__call__)
       13    0.000    0.000    0.000    0.000 init_ops_v2.py:749(__init__)
       13    0.000    0.000    0.000    0.000 init_ops_v2.py:994(_compute_fans)
       66    0.000    0.000    0.003    0.000 initializers.py:169(deserialize)
       66    0.001    0.000    0.001    0.000 initializers.py:176(<dictcomp>)
      142    0.000    0.000    0.004    0.000 initializers.py:189(get)
        3    0.000    0.000    0.007    0.002 input_layer.py:194(Input)
        3    0.000    0.000    0.007    0.002 input_layer.py:88(__init__)
      518    0.004    0.000    0.013    0.000 input_spec.py:132(assert_input_compatibility)
       38    0.000    0.000    0.000    0.000 input_spec.py:57(__init__)
       38    0.000    0.000    0.000    0.000 input_spec.py:75(<dictcomp>)
      145    0.002    0.000    0.008    0.000 inspect.py:1089(getfullargspec)
       35    0.000    0.000    0.000    0.000 inspect.py:1495(currentframe)
      482    0.000    0.000    0.000    0.000 inspect.py:159(isfunction)
       17    0.000    0.000    0.000    0.000 inspect.py:172(isgeneratorfunction)
      145    0.002    0.000    0.005    0.000 inspect.py:2102(_signature_from_function)
  230/145    0.001    0.000    0.005    0.000 inspect.py:2183(_signature_from_callable)
       66    0.000    0.000    0.000    0.000 inspect.py:229(istraceback)
       66    0.000    0.000    0.000    0.000 inspect.py:239(isframe)
      485    0.001    0.000    0.002    0.000 inspect.py:2452(__init__)
      970    0.000    0.000    0.000    0.000 inspect.py:2502(name)
      705    0.000    0.000    0.000    0.000 inspect.py:2506(default)
      485    0.000    0.000    0.000    0.000 inspect.py:2510(annotation)
      485    0.000    0.000    0.000    0.000 inspect.py:2514(kind)
       64    0.000    0.000    0.000    0.000 inspect.py:253(iscode)
      145    0.001    0.000    0.001    0.000 inspect.py:2732(__init__)
       38    0.000    0.000    0.000    0.000 inspect.py:277(isbuiltin)
      630    0.000    0.000    0.000    0.000 inspect.py:2781(<genexpr>)
      145    0.000    0.000    0.000    0.000 inspect.py:2817(parameters)
      145    0.000    0.000    0.000    0.000 inspect.py:2821(return_annotation)
       17    0.000    0.000    0.000    0.000 inspect.py:479(getmro)
        2    0.000    0.000    0.000    0.000 inspect.py:485(unwrap)
        2    0.000    0.000    0.000    0.000 inspect.py:502(_is_wrapper)
      172    0.000    0.000    0.000    0.000 inspect.py:64(ismodule)
       60    0.000    0.000    0.001    0.000 inspect.py:643(getfile)
       58    0.000    0.000    0.002    0.000 inspect.py:680(getsourcefile)
      123    0.000    0.000    0.000    0.000 inspect.py:687(<genexpr>)
      164    0.000    0.000    0.000    0.000 inspect.py:690(<genexpr>)
       17    0.000    0.000    0.000    0.000 inspect.py:702(getabsfile)
       99    0.000    0.000    0.001    0.000 inspect.py:714(getmodule)
      307    0.000    0.000    0.000    0.000 inspect.py:73(isclass)
        4    0.000    0.000    0.000    0.000 inspect.py:760(findsource)
      618    0.000    0.000    0.000    0.000 inspect.py:81(ismethod)
        4    0.000    0.000    0.000    0.000 inspect.py:882(__init__)
      282    0.000    0.000    0.000    0.000 inspect.py:891(tokeneater)
        4    0.000    0.000    0.002    0.000 inspect.py:935(getblock)
        2    0.000    0.000    0.001    0.001 inspect.py:946(getsourcelines)
        2    0.002    0.001    0.006    0.003 inspect_utils.py:116(_fix_linecache_record)
        2    0.000    0.000    0.008    0.004 inspect_utils.py:143(getimmediatesource)
        2    0.000    0.000    0.000    0.000 inspect_utils.py:151(getnamespace)
       51    0.000    0.000    0.000    0.000 inspect_utils.py:240(_get_unbound_function)
       17    0.000    0.000    0.000    0.000 inspect_utils.py:250(getdefiningclass)
       17    0.000    0.000    0.000    0.000 inspect_utils.py:265(getmethodclass)
        2    0.000    0.000    0.000    0.000 inspect_utils.py:339(getfutureimports)
        2    0.000    0.000    0.000    0.000 inspect_utils.py:350(<genexpr>)
        2    0.000    0.000    0.000    0.000 inspect_utils.py:60(islambda)
       51    0.000    0.000    0.000    0.000 inspect_utils.py:68(isnamedtuple)
       38    0.000    0.000    0.002    0.000 inspect_utils.py:82(isbuiltin)
     5928    0.001    0.000    0.001    0.000 inspect_utils.py:84(<genexpr>)
       38    0.000    0.000    0.000    0.000 inspect_utils.py:96(isconstructor)
       36    0.000    0.000    0.003    0.000 iostream.py:195(schedule)
       11    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
       11    0.000    0.000    0.001    0.000 iostream.py:320(_schedule_flush)
        8    0.000    0.000    0.009    0.001 iostream.py:334(flush)
       11    0.000    0.000    0.002    0.000 iostream.py:382(write)
       36    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
  521/483    0.002    0.000    0.006    0.000 layer_utils.py:126(wrapped)
       44    0.000    0.000    0.000    0.000 layer_utils.py:161(mark_as)
      521    0.000    0.000    0.000    0.000 layer_utils.py:166(in_cached_state)
       51    0.000    0.000    0.000    0.000 layer_utils.py:179(__init__)
       48    0.000    0.000    1.716    0.036 layer_utils.py:194(add_parent)
      521    0.001    0.000    0.001    0.000 layer_utils.py:205(get)
    44/41    0.000    0.000    0.000    0.000 layer_utils.py:209(_set)
       41    0.000    0.000    0.000    0.000 layer_utils.py:216(mark_cached)
        3    0.000    0.000    0.000    0.000 layer_utils.py:220(invalidate)
    68/58    0.000    0.000    0.001    0.000 layer_utils.py:224(invalidate_all)
1246/1128    0.002    0.000    0.008    0.000 layer_utils.py:234(filter_empty_layer_containers)
      798    0.000    0.000    0.001    0.000 layer_utils.py:37(is_layer)
      912    0.001    0.000    0.001    0.000 layer_utils.py:43(has_weights)
        4    0.000    0.000    0.000    0.000 linecache.py:37(getlines)
        4    0.000    0.000    0.000    0.000 linecache.py:53(checkcache)
       22    0.000    0.000    0.000    0.000 liveness.py:106(__init__)
       22    0.000    0.000    0.029    0.001 liveness.py:112(visit_FunctionDef)
       22    0.000    0.000    0.000    0.000 liveness.py:155(__init__)
  2407/22    0.002    0.000    0.029    0.001 liveness.py:160(visit)
       22    0.000    0.000    0.028    0.001 liveness.py:170(visit_FunctionDef)
       16    0.000    0.000    0.000    0.000 liveness.py:178(_block_statement_live_out)
       34    0.000    0.000    0.000    0.000 liveness.py:186(_block_statement_live_in)
        8    0.000    0.000    0.003    0.000 liveness.py:213(visit_Try)
        8    0.000    0.000    0.001    0.000 liveness.py:218(visit_ExceptHandler)
       18    0.000    0.000    0.020    0.001 liveness.py:223(visit_With)
        8    0.000    0.000    0.000    0.000 liveness.py:227(visit_Expr)
       22    0.000    0.000    0.058    0.003 liveness.py:235(resolve)
       22    0.000    0.000    0.000    0.000 liveness.py:43(__init__)
      346    0.000    0.000    0.000    0.000 liveness.py:49(init_state)
      241    0.001    0.000    0.002    0.000 liveness.py:52(visit_node)
        2    0.000    0.000    0.002    0.001 loader.py:36(load_source)
        2    0.000    0.000    0.020    0.010 loader.py:56(load_ast)
     2484    0.004    0.000    0.008    0.000 lock_util.py:106(_another_group_active)
     4968    0.002    0.000    0.002    0.000 lock_util.py:108(<genexpr>)
     7452    0.003    0.000    0.003    0.000 lock_util.py:110(_validate_group_id)
     2484    0.001    0.000    0.001    0.000 lock_util.py:119(__init__)
     2484    0.002    0.000    0.018    0.000 lock_util.py:123(__enter__)
     2484    0.002    0.000    0.026    0.000 lock_util.py:126(__exit__)
       60    0.000    0.000    0.001    0.000 lock_util.py:54(__init__)
     2484    0.004    0.000    0.006    0.000 lock_util.py:74(group)
     2484    0.007    0.000    0.016    0.000 lock_util.py:86(acquire)
     2484    0.008    0.000    0.023    0.000 lock_util.py:96(release)
        1    0.000    0.000    0.000    0.000 logger.py:24(info)
        2    0.000    0.000    0.003    0.002 logical_expressions.py:135(transform)
       41    0.000    0.000    0.000    0.000 loss_scale.py:421(get)
       41    0.000    0.000    0.000    0.000 loss_scale.py:46(get)
   306094    1.668    0.000   27.875    0.000 math_ops.py:1007(r_binary_op_wrapper)
   276694    2.108    0.000   13.183    0.000 math_ops.py:1074(_truediv_python3)
   348028    0.706    0.000   15.197    0.000 math_ops.py:1271(_add_dispatch)
   174027    0.293    0.000    5.572    0.000 math_ops.py:1279(_mul_dispatch)
       10    0.000    0.000    0.009    0.001 math_ops.py:1360(logical_and)
   109860    0.435    0.000    5.735    0.000 math_ops.py:1484(tensor_equals)
       92    0.000    0.000    0.002    0.000 math_ops.py:732(cast)
   697440    3.196    0.000   37.093    0.000 math_ops.py:981(binary_op_wrapper)
     1668    0.001    0.000    0.002    0.000 memory.py:24(dismantle_ordered_dict)
      392    0.000    0.000    0.000    0.000 module_wrapper.py:154(__getattribute__)
        3    0.000    0.000    0.000    0.000 monitoring.py:134(get_cell)
        3    0.000    0.000    0.000    0.000 monitoring.py:293(__init__)
        3    0.000    0.000    0.000    0.000 monitoring.py:301(set)
        3    0.000    0.000    0.000    0.000 monitoring.py:333(get_cell)
        2    0.000    0.000    0.000    0.000 multiarray.py:1043(copyto)
      297    0.000    0.000    0.000    0.000 multiarray.py:145(concatenate)
       10    0.000    0.000    0.000    0.000 naming.py:103(new_symbol)
        4    0.000    0.000    0.000    0.000 naming.py:35(__init__)
        2    0.000    0.000    0.000    0.000 naming.py:39(_as_symbol_name)
        2    0.000    0.000    0.000    0.000 naming.py:90(function_name)
    30816    0.015    0.000    0.081    0.000 nest.py:104(_is_namedtuple)
      151    0.000    0.000    0.002    0.000 nest.py:1288(yield_flat_paths)
    15954    0.082    0.000    0.141    0.000 nest.py:129(_sequence_like)
      116    0.000    0.000    0.003    0.000 nest.py:1355(flatten_with_tuple_paths)
    76124    0.038    0.000    0.302    0.000 nest.py:191(_yield_value)
    76426    0.073    0.000    0.265    0.000 nest.py:196(_yield_sorted_items)
   203612    0.109    0.000    0.580    0.000 nest.py:259(flatten)
      136    0.000    0.000    0.001    0.000 nest.py:341(assert_same_structure)
    15918    0.075    0.000    0.409    0.000 nest.py:444(_packed_nest_with_indices)
    17653    0.056    0.000    0.636    0.000 nest.py:479(_pack_sequence_as)
    17653    0.016    0.000    0.652    0.000 nest.py:519(pack_sequence_as)
     2181    0.008    0.000    0.142    0.000 nest.py:555(map_structure)
     2181    0.001    0.000    0.005    0.000 nest.py:613(<listcomp>)
     2181    0.002    0.000    0.113    0.000 nest.py:617(<listcomp>)
  221/151    0.000    0.000    0.002    0.000 nest.py:702(_yield_flat_up_to)
     1288    0.001    0.000    0.002    0.000 nest.py:96(_sorted)
        3    0.000    0.000    0.000    0.000 network.py:1356(_validate_graph_inputs_and_outputs)
        3    0.000    0.000    0.000    0.000 network.py:1359(<setcomp>)
        3    0.000    0.000    0.000    0.000 network.py:1394(<listcomp>)
        3    0.000    0.000    0.000    0.000 network.py:1499(_compute_tensor_usage_count)
        6    0.000    0.000    0.000    0.000 network.py:1507(<genexpr>)
       35    0.000    0.000    0.000    0.000 network.py:1515(<setcomp>)
        3    0.000    0.000    0.000    0.000 network.py:1584(_set_save_spec)
       38    0.000    0.000    0.000    0.000 network.py:1620(_make_node_key)
        3    0.000    0.000    0.002    0.001 network.py:1624(_map_graph_network)
     40/5    0.000    0.000    0.001    0.000 network.py:1649(build_map)
        3    0.000    0.000    0.007    0.002 network.py:167(__init__)
       38    0.000    0.000    0.000    0.000 network.py:1766(<lambda>)
        3    0.000    0.000    0.000    0.000 network.py:1799(<listcomp>)
        3    0.000    0.000    0.001    0.000 network.py:193(_base_init)
        3    0.000    0.000    0.006    0.002 network.py:229(_init_graph_network)
        8    0.000    0.000    0.000    0.000 network.py:253(<genexpr>)
        3    0.000    0.000    0.000    0.000 network.py:262(<lambda>)
        3    0.000    0.000    0.000    0.000 network.py:346(_set_output_names)
        3    0.000    0.000    0.001    0.000 network.py:378(dynamic)
       41    0.000    0.000    0.000    0.000 network.py:382(<genexpr>)
  471/387    0.001    0.000    0.004    0.000 network.py:438(__setattr__)
      210    0.000    0.000    0.000    0.000 network.py:444(<genexpr>)
        3    0.000    0.000    0.000    0.000 network.py:512(_should_compute_mask)
        6    0.000    0.000    0.001    0.000 network.py:527(layers)
       36    0.000    0.000    0.000    0.000 network.py:584(input_spec)
       36    0.000    0.000    1.206    0.033 network.py:695(call)
       36    0.008    0.000    1.206    0.033 network.py:800(_run_internal_graph)
       36    0.000    0.000    0.000    0.000 network.py:828(<listcomp>)
      824    0.001    0.000    0.001    0.000 network.py:853(<genexpr>)
      412    0.001    0.000    0.001    0.000 network.py:858(<lambda>)
      108    0.000    0.000    0.000    0.000 network.py:877(_map_tensor_if_from_keras_layer)
       36    0.000    0.000    0.000    0.000 network.py:905(<listcomp>)
       36    0.000    0.000    0.000    0.000 network.py:918(_flatten_to_reference_inputs)
       36    0.000    0.000    0.005    0.000 network.py:931(_conform_to_reference_input)
      128    0.001    0.000    0.618    0.005 nn_impl.py:1425(fused_batch_norm)
       10    0.000    0.000    0.004    0.000 nn_ops.py:1021(__init__)
       10    0.000    0.000    0.001    0.000 nn_ops.py:1084(_build_op)
      118    0.000    0.000    0.209    0.002 nn_ops.py:1093(__call__)
      118    0.000    0.000    0.000    0.000 nn_ops.py:1531(_convert_padding)
       10    0.000    0.000    0.001    0.000 nn_ops.py:161(__init__)
      118    0.001    0.000    0.208    0.002 nn_ops.py:1920(conv2d)
      118    0.000    0.000    0.209    0.002 nn_ops.py:230(__call__)
       56    0.000    0.000    0.000    0.000 nn_ops.py:2725(_tf_deterministic_ops)
       56    0.001    0.000    0.088    0.002 nn_ops.py:2738(bias_add)
       10    0.000    0.000    0.002    0.000 nn_ops.py:503(__init__)
       30    0.000    0.000    0.000    0.000 nn_ops.py:533(<genexpr>)
       30    0.000    0.000    0.000    0.000 nn_ops.py:534(<genexpr>)
      236    0.000    0.000    0.001    0.000 nn_ops.py:60(_get_sequence)
      118    0.000    0.000    0.209    0.002 nn_ops.py:637(__call__)
       10    0.000    0.000    0.001    0.000 nn_ops.py:720(_get_strides_and_dilation_rate)
       41    0.000    0.000    0.000    0.000 node.py:117(<listcomp>)
       76    0.000    0.000    0.001    0.000 node.py:134(iterate_inbound)
       38    0.000    0.000    0.000    0.000 node.py:153(<listcomp>)
       38    0.000    0.000    0.000    0.000 node.py:168(_get_all_node_dependencies)
       41    0.000    0.000    0.003    0.000 node.py:66(__init__)
       10    0.000    0.000    0.012    0.001 normalization.py:176(__init__)
       10    0.000    0.000    0.000    0.000 normalization.py:249(_raise_if_fused_cannot_be_used)
       10    0.000    0.000    0.000    0.000 normalization.py:274(_fused_can_be_used)
      236    0.000    0.000    0.000    0.000 normalization.py:281(trainable)
       10    0.000    0.000    0.000    0.000 normalization.py:285(trainable)
       10    0.000    0.000    0.012    0.001 normalization.py:291(_get_trainable_var)
       40    0.000    0.000    0.001    0.000 normalization.py:297(_param_dtype)
      118    0.000    0.000    0.001    0.000 normalization.py:305(_support_zero_size_input)
       10    0.001    0.000    1.764    0.176 normalization.py:310(build)
       10    0.000    0.000    0.000    0.000 normalization.py:372(<dictcomp>)
      118    0.001    0.000    0.837    0.007 normalization.py:523(_fused_batch_norm)
      128    0.000    0.000    0.000    0.000 normalization.py:550(_maybe_add_or_remove_bessels_correction)
       10    0.000    0.000    0.053    0.005 normalization.py:568(_fused_batch_norm_training)
      118    0.000    0.000    0.565    0.005 normalization.py:584(_fused_batch_norm_inference)
       10    0.000    0.000    0.000    0.000 normalization.py:611(<lambda>)
       10    0.000    0.000    0.000    0.000 normalization.py:612(<lambda>)
      118    0.000    0.000    0.022    0.000 normalization.py:710(_get_training_value)
      118    0.001    0.000    0.860    0.007 normalization.py:724(call)
       22    0.000    0.000    0.000    0.000 numeric.py:1786(isscalar)
        2    0.000    0.000    0.001    0.000 numeric.py:283(full)
      594    0.001    0.000    0.002    0.000 numerictypes.py:293(issubclass_)
      297    0.001    0.000    0.003    0.000 numerictypes.py:365(issubdtype)
     1312    0.000    0.000    0.000    0.000 object_identity.py:125(__init__)
     2406    0.001    0.000    0.002    0.000 object_identity.py:128(_wrap_key)
     1531    0.002    0.000    0.004    0.000 object_identity.py:131(__getitem__)
      669    0.001    0.000    0.001    0.000 object_identity.py:134(__setitem__)
      206    0.000    0.000    0.001    0.000 object_identity.py:137(__delitem__)
     6072    0.007    0.000    0.008    0.000 object_identity.py:174(__init__)
     6227    0.001    0.000    0.001    0.000 object_identity.py:175(<genexpr>)
     3946    0.002    0.000    0.003    0.000 object_identity.py:183(_wrap_key)
     1741    0.001    0.000    0.003    0.000 object_identity.py:186(__contains__)
      200    0.000    0.000    0.000    0.000 object_identity.py:189(discard)
     1940    0.002    0.000    0.008    0.000 object_identity.py:192(add)
     7423    0.006    0.000    0.020    0.000 object_identity.py:195(update)
     7423    0.003    0.000    0.011    0.000 object_identity.py:196(<listcomp>)
      313    0.000    0.000    0.000    0.000 object_identity.py:208(__len__)
    11767    0.006    0.000    0.006    0.000 object_identity.py:211(__iter__)
      832    0.001    0.000    0.003    0.000 object_identity.py:220(_wrap_key)
       58    0.000    0.000    0.001    0.000 object_identity.py:223(__len__)
       58    0.000    0.000    0.001    0.000 object_identity.py:225(<listcomp>)
     1568    0.001    0.000    0.001    0.000 object_identity.py:227(__iter__)
     7184    0.002    0.000    0.002    0.000 object_identity.py:35(__init__)
     1750    0.000    0.000    0.000    0.000 object_identity.py:38(unwrapped)
      947    0.000    0.000    0.000    0.000 object_identity.py:42(_assert_type)
      947    0.000    0.000    0.001    0.000 object_identity.py:54(__eq__)
     7184    0.002    0.000    0.003    0.000 object_identity.py:63(__hash__)
      832    0.001    0.000    0.002    0.000 object_identity.py:75(__init__)
     1452    0.000    0.000    0.000    0.000 object_identity.py:78(unwrapped)
     2382    0.004    0.000    0.008    0.000 op_callbacks.py:118(should_invoke_op_callbacks)
      256    0.000    0.000    0.001    0.000 op_def_library.py:141(_MakeFloat)
      944    0.000    0.000    0.001    0.000 op_def_library.py:148(_MakeInt)
      420    0.000    0.000    0.001    0.000 op_def_library.py:159(_MakeStr)
      359    0.000    0.000    0.000    0.000 op_def_library.py:166(_MakeBool)
     1825    0.005    0.000    0.012    0.000 op_def_library.py:173(_MakeType)
       53    0.000    0.000    0.001    0.000 op_def_library.py:184(_MakeShape)
       40    0.000    0.000    0.000    0.000 op_def_library.py:216(_MakeFunc)
     3124    0.001    0.000    0.001    0.000 op_def_library.py:235(_MaybeColocateWith)
1562/1132    0.106    0.000    1.223    0.001 op_def_library.py:299(_apply_op_helper)
     1039    0.002    0.000    0.002    0.000 op_def_library.py:38(_Attr)
       20    0.000    0.000    0.000    0.000 op_def_library.py:447(<listcomp>)
     2192    0.001    0.000    0.001    0.000 op_def_library.py:46(_AttrValue)
     2393    0.002    0.000    0.006    0.000 op_def_library.py:510(<listcomp>)
     2864    0.006    0.000    0.011    0.000 op_def_library.py:53(_SatisfiesTypeConstraint)
     2393    0.002    0.000    0.002    0.000 op_def_library.py:64(_IsListParameter)
      374    0.000    0.000    0.001    0.000 op_def_library.py:682(<listcomp>)
       40    0.000    0.000    0.000    0.000 op_def_library.py:695(<listcomp>)
       20    0.000    0.000    0.001    0.000 op_def_library.py:700(<listcomp>)
     1562    0.002    0.000    0.002    0.000 op_def_library.py:737(<listcomp>)
     5095    0.002    0.000    0.004    0.000 op_def_library.py:80(_IsListValue)
     1562    0.002    0.000    0.009    0.000 op_def_library.py:84(_Flatten)
     1562    0.002    0.000    0.006    0.000 op_def_library.py:87(<listcomp>)
     1562    0.001    0.000    0.001    0.000 op_def_library.py:89(<listcomp>)
     4043    0.002    0.000    0.002    0.000 op_def_registry.py:34(get)
     1082    0.000    0.000    0.000    0.000 ops.py:102(tensor_id)
     1170    0.002    0.000    0.010    0.000 ops.py:1058(shape)
       20    0.000    0.000    0.000    0.000 ops.py:1071(get_shape)
   110154    0.181    0.000    0.181    0.000 ops.py:1115(graph)
   174602    0.205    0.000    4.781    0.000 ops.py:1220(convert_to_tensor_v2)
1171640/1170898    4.081    0.000   17.454    0.000 ops.py:1290(convert_to_tensor)
       20    0.000    0.000    0.015    0.001 ops.py:1365(internal_convert_n_to_tensor)
      151    0.000    0.000    0.010    0.000 ops.py:1444(convert_to_tensor_or_composite)
      151    0.000    0.000    0.010    0.000 ops.py:1467(internal_convert_to_tensor_or_composite)
  1280595    0.200    0.000    0.200    0.000 ops.py:156(__init__)
     2484    0.036    0.000    0.058    0.000 ops.py:1579(_NodeDef)
  1280595    0.159    0.000    0.159    0.000 ops.py:159(__enter__)
     2484    0.035    0.000    0.201    0.000 ops.py:1606(_create_c_op)
  1280595    0.221    0.000    0.221    0.000 ops.py:162(__exit__)
       20    0.000    0.000    0.000    0.000 ops.py:1636(<listcomp>)
     2484    0.050    0.000    0.455    0.000 ops.py:1681(__init__)
      102    0.000    0.000    0.000    0.000 ops.py:1764(<listcomp>)
     4805    0.003    0.000    0.012    0.000 ops.py:1767(<genexpr>)
     2484    0.004    0.000    0.010    0.000 ops.py:1837(_control_flow_post_processing)
     7327    0.001    0.000    0.001    0.000 ops.py:1878(_get_control_flow_context)
       20    0.000    0.000    0.000    0.000 ops.py:1894(name)
     2491    0.001    0.000    0.001    0.000 ops.py:2087(_add_control_inputs)
      357    0.000    0.000    0.001    0.000 ops.py:209(is_dense_tensor_like)
     4987    0.001    0.000    0.001    0.000 ops.py:2149(outputs)
     3798    0.007    0.000    0.023    0.000 ops.py:2154(inputs)
       80    0.000    0.000    0.000    0.000 ops.py:2198(_control_outputs)
       80    0.000    0.000    0.000    0.000 ops.py:2213(<listcomp>)
    32643    0.011    0.000    0.032    0.000 ops.py:2218(type)
    10406    0.002    0.000    0.002    0.000 ops.py:2223(graph)
       58    0.000    0.000    0.002    0.000 ops.py:2264(_set_attr)
       58    0.000    0.000    0.001    0.000 ops.py:2273(_set_attr_with_buf)
     1759    0.009    0.000    0.137    0.000 ops.py:2313(get_attr)
     3318    0.003    0.000    0.007    0.000 ops.py:255(uid)
     1695    0.001    0.000    0.001    0.000 ops.py:2643(name_from_scope_name)
       60    0.002    0.000    0.005    0.000 ops.py:2704(__init__)
       36    0.000    0.000    0.000    0.000 ops.py:2821(_variable_creator_scope)
       18    0.000    0.000    0.000    0.000 ops.py:2845(<lambda>)
     4312    0.002    0.000    0.003    0.000 ops.py:2857(_variable_creator_stack)
     4238    0.001    0.000    0.001    0.000 ops.py:2881(_variable_creator_stack)
     5008    0.001    0.000    0.001    0.000 ops.py:2885(_check_not_finalized)
     2484    0.010    0.000    0.012    0.000 ops.py:2894(_add_op)
     5016    0.002    0.000    0.002    0.000 ops.py:2913(_c_graph)
       40    0.000    0.000    0.000    0.000 ops.py:2954(seed)
       60    0.000    0.000    0.000    0.000 ops.py:2959(seed)
     3366    0.001    0.000    0.001    0.000 ops.py:2990(_get_control_flow_context)
     1604    0.000    0.000    0.000    0.000 ops.py:2998(_set_control_flow_context)
       40    0.000    0.000    0.000    0.000 ops.py:3139(_is_function)
       40    0.000    0.000    0.002    0.000 ops.py:3161(_add_function)
    21467    0.006    0.000    0.006    0.000 ops.py:3196(building_function)
     2484    0.029    0.000    0.682    0.000 ops.py:3260(_create_op_internal)
     4907    0.002    0.000    0.002    0.000 ops.py:3314(<genexpr>)
     2484    0.022    0.000    0.070    0.000 ops.py:3363(_create_op_helper)
      828    0.007    0.000    0.007    0.000 ops.py:3603(get_operations)
     2583    0.003    0.000    0.003    0.000 ops.py:3642(_get_operation_by_name_unsafe)
     2423    0.002    0.000    0.007    0.000 ops.py:3665(_get_operation_by_tf_operation)
     2423    0.005    0.000    0.012    0.000 ops.py:3690(_get_tensor_by_tf_output)
      922    0.002    0.000    0.005    0.000 ops.py:3710(_get_op_def)
     2119    0.002    0.000    0.008    0.000 ops.py:3728(as_default)
      100    0.000    0.000    0.000    0.000 ops.py:3814(get_collection_ref)
       58    0.000    0.000    0.000    0.000 ops.py:3839(get_collection)
       20    0.000    0.000    0.000    0.000 ops.py:3879(get_all_collection_keys)
       20    0.000    0.000    0.000    0.000 ops.py:3882(<listcomp>)
     3144    0.011    0.000    0.032    0.000 ops.py:389(__init__)
     9937    0.004    0.000    0.006    0.000 ops.py:3921(_name_stack)
     4244    0.002    0.000    0.002    0.000 ops.py:3928(_name_stack)
     4244    0.012    0.000    0.033    0.000 ops.py:3933(name_scope)
     2967    0.008    0.000    0.014    0.000 ops.py:4048(unique_name)
       99    0.000    0.000    0.000    0.000 ops.py:4100(get_name_scope)
     1484    0.002    0.000    0.012    0.000 ops.py:4117(_colocate_with_for_gradient)
     1484    0.004    0.000    0.008    0.000 ops.py:4130(colocate_with)
     3144    0.004    0.000    0.036    0.000 ops.py:416(_create_with_tf_output)
     8725    0.002    0.000    0.002    0.000 ops.py:422(op)
    10230    0.002    0.000    0.002    0.000 ops.py:427(dtype)
     2484    0.007    0.000    0.020    0.000 ops.py:4281(_apply_device_functions)
     4599    0.003    0.000    0.004    0.000 ops.py:432(graph)
      822    0.001    0.000    0.001    0.000 ops.py:4364(__init__)
      822    0.003    0.000    0.006    0.000 ops.py:4399(__enter__)
      822    0.002    0.000    0.005    0.000 ops.py:4409(__exit__)
      822    0.000    0.000    0.000    0.000 ops.py:4417(control_inputs)
      822    0.001    0.000    0.001    0.000 ops.py:4421(add_op)
       70    0.000    0.000    0.000    0.000 ops.py:4426(op_in_group)
      822    0.001    0.000    0.001    0.000 ops.py:4431(_push_control_dependencies_controller)
      822    0.001    0.000    0.003    0.000 ops.py:4434(_pop_control_dependencies_controller)
       20    0.000    0.000    0.000    0.000 ops.py:4438(_current_control_dependencies)
     2484    0.003    0.000    0.007    0.000 ops.py:4445(_control_dependencies_for_inputs)
      822    0.000    0.000    0.000    0.000 ops.py:4476(<genexpr>)
     2484    0.002    0.000    0.007    0.000 ops.py:4479(_record_op_seen_by_control_dependencies)
      822    0.001    0.000    0.002    0.000 ops.py:4488(control_dependencies)
     2178    0.001    0.000    0.013    0.000 ops.py:451(shape)
      112    0.001    0.000    0.001    0.000 ops.py:4602(_attr_scope)
        2    0.000    0.000    0.000    0.000 ops.py:4804(prevent_feeding)
       20    0.000    0.000    0.000    0.000 ops.py:4812(prevent_fetching)
       60    0.000    0.000    0.000    0.000 ops.py:4823(switch_to_thread_local)
    13607    0.007    0.000    0.010    0.000 ops.py:4840(_device_function_stack)
     2484    0.002    0.000    0.005    0.000 ops.py:4861(_snapshot_device_function_stack_metadata)
     3603    0.002    0.000    0.002    0.000 ops.py:4877(_device_function_stack)
     5750    0.003    0.000    0.004    0.000 ops.py:4886(_colocation_stack)
     2484    0.004    0.000    0.007    0.000 ops.py:4901(_snapshot_colocation_stack_metadata)
     2484    0.001    0.000    0.001    0.000 ops.py:4903(<dictcomp>)
     1524    0.001    0.000    0.001    0.000 ops.py:4908(_colocation_stack)
     8256    0.005    0.000    0.009    0.000 ops.py:4917(_control_dependencies_stack)
     1604    0.001    0.000    0.001    0.000 ops.py:4929(_control_dependencies_stack)
    21981    0.021    0.000    0.033    0.000 ops.py:4936(_distribution_strategy_stack)
     4238    0.002    0.000    0.002    0.000 ops.py:4943(_distribution_strategy_stack)
     4721    0.002    0.000    0.003    0.000 ops.py:4959(_auto_cast_variable_read_dtype)
     5204    0.004    0.000    0.007    0.000 ops.py:4974(_auto_cast_variable_read_dtype)
      966    0.001    0.000    0.003    0.000 ops.py:4980(_enable_auto_casting_variables)
     2484    0.003    0.000    0.009    0.000 ops.py:5002(_mutation_lock)
      561    0.002    0.000    0.012    0.000 ops.py:502(_c_api_shape)
        1    0.000    0.000    0.000    0.000 ops.py:5021(device)
      551    0.000    0.000    0.000    0.000 ops.py:510(<listcomp>)
      742    0.002    0.000    0.007    0.000 ops.py:5110(_colocate_with_for_gradient)
      742    0.000    0.000    0.007    0.000 ops.py:5133(colocate_with)
      921    0.002    0.000    0.007    0.000 ops.py:5144(control_dependencies)
    65499    0.040    0.000    0.050    0.000 ops.py:5184(get_default)
     4238    0.004    0.000    0.006    0.000 ops.py:5201(get_controller)
    65499    0.096    0.000    0.161    0.000 ops.py:5371(get_default)
    45139    0.015    0.000    0.015    0.000 ops.py:5378(_GetGlobalDefaultGraph)
     4238    0.018    0.000    0.058    0.000 ops.py:5391(get_controller)
     1596    0.008    0.000    0.015    0.000 ops.py:5412(_get_outer_context_and_inner_device_stack)
      640    0.002    0.000    0.010    0.000 ops.py:5450(init_scope)
   111401    0.128    0.000    0.479    0.000 ops.py:5555(executing_eagerly_outside_functions)
      953    0.001    0.000    0.003    0.000 ops.py:5588(inside_function)
    65499    0.053    0.000    0.214    0.000 ops.py:5800(get_default_graph)
        1    0.000    0.000    0.000    0.000 ops.py:5824(get_name_scope)
     1562    0.002    0.000    0.006    0.000 ops.py:5859(_get_graph_from_inputs)
      122    0.000    0.000    0.000    0.000 ops.py:593(get_shape)
       75    0.001    0.000    0.001    0.000 ops.py:597(set_shape)
      544    0.306    0.001    0.539    0.001 ops.py:6075(dismantle_graph)
       58    0.000    0.000    0.000    0.000 ops.py:6155(get_collection)
  1326824    2.710    0.000    4.137    0.000 ops.py:6189(name_scope)
    44689    0.045    0.000    0.045    0.000 ops.py:6227(<genexpr>)
     1639    0.002    0.000    0.002    0.000 ops.py:6243(__init__)
     1639    0.003    0.000    0.034    0.000 ops.py:6262(__enter__)
     1639    0.002    0.000    0.007    0.000 ops.py:6302(__exit__)
    44844    0.095    0.000    0.155    0.000 ops.py:6360(enter_eager_name_scope)
    45327    0.060    0.000    0.067    0.000 ops.py:6405(__init__)
    45327    0.102    0.000    0.294    0.000 ops.py:6423(__enter__)
    44844    0.027    0.000    0.083    0.000 ops.py:6437(<lambda>)
    45327    0.062    0.000    0.173    0.000 ops.py:6444(__exit__)
      742    0.000    0.000    0.000    0.000 ops.py:6579(_op_to_colocate_with)
      294    0.000    0.000    0.001    0.000 ops.py:6604(_is_keras_symbolic_tensor)
      220    0.000    0.000    0.001    0.000 ops.py:665(consumers)
     2484    0.010    0.000    0.011    0.000 ops.py:6690(_reconstruct_sequence_inputs)
       20    0.000    0.000    0.001    0.000 ops.py:6747(set_int_list_attr)
      220    0.000    0.000    0.000    0.000 ops.py:675(<listcomp>)
     5788    0.002    0.000    0.002    0.000 ops.py:699(_as_tf_output)
   465362    1.969    0.000   58.337    0.000 ops.py:883(__bool__)
   480366    0.521    0.000   60.015    0.000 ops.py:924(_numpy)
  1998757    1.248    0.000    1.677    0.000 ops.py:931(dtype)
    15004    0.053    0.000    3.760    0.000 ops.py:937(numpy)
        8    0.000    0.000    0.000    0.000 origin_info.py:162(__init__)
        4    0.000    0.000    0.000    0.000 origin_info.py:169(__init__)
      307    0.000    0.000    0.000    0.000 origin_info.py:191(_absolute_lineno)
      307    0.000    0.000    0.000    0.000 origin_info.py:194(_absolute_col_offset)
      307    0.001    0.000    0.002    0.000 origin_info.py:197(_attach_origin_info)
    522/4    0.001    0.000    0.005    0.001 origin_info.py:212(visit)
        4    0.000    0.000    0.009    0.002 origin_info.py:226(resolve)
        2    0.000    0.000    0.003    0.002 origin_info.py:263(resolve_entity)
      118    0.000    0.000    0.000    0.000 origin_info.py:57(line_loc)
        2    0.001    0.000    0.015    0.008 origin_info.py:89(create_source_map)
      271    0.001    0.000    0.001    0.000 os.py:664(__getitem__)
      271    0.000    0.000    0.000    0.000 os.py:742(encode)
      270    0.000    0.000    0.002    0.000 os.py:760(getenv)
        2    0.000    0.000    0.002    0.001 parser.py:119(_attempt_to_parse_normal_source)
        2    0.000    0.000    0.010    0.005 parser.py:182(parse_entity)
        2    0.000    0.000    0.000    0.000 parser.py:209(<genexpr>)
       35    0.000    0.000    0.012    0.000 parser.py:220(parse)
       13    0.000    0.000    0.001    0.000 parser.py:244(parse_expression)
        2    0.000    0.000    0.003    0.002 parser.py:263(unparse)
        2    0.000    0.000    0.000    0.000 parser.py:46(_unfold_continuations)
        2    0.000    0.000    0.001    0.001 parser.py:51(dedent_block)
       41    0.000    0.000    0.001    0.000 policy.py:302(__init__)
       41    0.000    0.000    0.000    0.000 policy.py:351(_parse_name)
      214    0.000    0.000    0.000    0.000 policy.py:391(variable_dtype)
     1073    0.000    0.000    0.000    0.000 policy.py:406(compute_dtype)
       16    0.000    0.000    0.000    0.000 policy.py:434(should_cast_variables)
       41    0.000    0.000    0.000    0.000 policy.py:454(name)
       38    0.000    0.000    0.001    0.000 policy.py:489(global_policy)
       38    0.000    0.000    0.000    0.000 policy.py:512(policy_defaults_to_floatx)
        1    0.000    0.000    0.000    0.000 posixpath.py:121(splitext)
        3    0.000    0.000    0.000    0.000 posixpath.py:144(basename)
        1    0.000    0.000    0.000    0.000 posixpath.py:154(dirname)
        3    0.000    0.000    0.000    0.000 posixpath.py:338(normpath)
        3    0.000    0.000    0.000    0.000 posixpath.py:376(abspath)
       11    0.000    0.000    0.000    0.000 posixpath.py:41(_get_sep)
        3    0.000    0.000    0.000    0.000 posixpath.py:64(isabs)
        4    0.000    0.000    0.000    0.000 posixpath.py:75(join)
        1    0.000    0.000    0.000    0.000 process.py:146(name)
        1    0.000    0.000    0.000    0.000 process.py:35(current_process)
        2    0.000    0.000    0.000    0.000 profiler.py:87(stop)
     2056    0.000    0.000    0.000    0.000 qual_names.py:112(has_subscript)
     2056    0.000    0.000    0.000    0.000 qual_names.py:115(has_attr)
8615/8445    0.003    0.000    0.004    0.000 qual_names.py:163(__hash__)
 1016/986    0.001    0.000    0.002    0.000 qual_names.py:166(__eq__)
    24/12    0.000    0.000    0.000    0.000 qual_names.py:171(__str__)
      773    0.001    0.000    0.010    0.000 qual_names.py:225(visit_Name)
      246    0.001    0.000    0.007    0.000 qual_names.py:230(visit_Attribute)
       48    0.000    0.000    0.023    0.000 qual_names.py:260(resolve)
     1041    0.001    0.000    0.001    0.000 qual_names.py:68(__init__)
       16    0.000    0.000    0.000    0.000 random.py:223(_randbelow)
       16    0.000    0.000    0.000    0.000 random.py:255(choice)
        3    0.000    0.000    0.000    0.000 random.py:680(getrandbits)
       13    0.000    0.000    0.004    0.000 random_ops.py:204(random_uniform)
       13    0.000    0.000    0.000    0.000 random_seed.py:41(get_seed)
       26    0.000    0.000    0.000    0.000 re.py:169(match)
       92    0.000    0.000    0.001    0.000 re.py:184(sub)
      985    0.000    0.000    0.001    0.000 re.py:231(compile)
     1103    0.000    0.000    0.000    0.000 re.py:286(_compile)
       76    0.000    0.000    0.000    0.000 re.py:324(_subx)
       20    0.000    0.000    0.000    0.000 re.py:330(filter)
      173    0.000    0.000    0.002    0.000 reaching_definitions.py:101(__sub__)
       22    0.000    0.000    0.001    0.000 reaching_definitions.py:115(__init__)
      346    0.000    0.000    0.000    0.000 reaching_definitions.py:124(init_state)
      173    0.002    0.000    0.012    0.000 reaching_definitions.py:127(visit_node)
       22    0.000    0.000    0.000    0.000 reaching_definitions.py:206(__init__)
       22    0.000    0.000    0.037    0.002 reaching_definitions.py:213(visit_FunctionDef)
      524    0.001    0.000    0.004    0.000 reaching_definitions.py:245(visit_Name)
       16    0.000    0.000    0.000    0.000 reaching_definitions.py:267(_aggregate_predecessors_defined_in)
        8    0.000    0.000    0.002    0.000 reaching_definitions.py:298(visit_Try)
        8    0.000    0.000    0.001    0.000 reaching_definitions.py:302(visit_ExceptHandler)
  1819/22    0.002    0.000    0.038    0.002 reaching_definitions.py:308(visit)
       22    0.000    0.000    0.038    0.002 reaching_definitions.py:320(resolve)
      201    0.000    0.000    0.000    0.000 reaching_definitions.py:52(__init__)
     1189    0.001    0.000    0.003    0.000 reaching_definitions.py:69(__init__)
      497    0.001    0.000    0.001    0.000 reaching_definitions.py:72(<dictcomp>)
      135    0.000    0.000    0.001    0.000 reaching_definitions.py:76(<dictcomp>)
      173    0.001    0.000    0.001    0.000 reaching_definitions.py:82(__eq__)
      173    0.000    0.000    0.001    0.000 reaching_definitions.py:88(__ne__)
      324    0.002    0.000    0.004    0.000 reaching_definitions.py:91(__or__)
        1    0.000    0.000    0.000    0.000 registration.py:102(spec)
        1    0.000    0.000    0.191    0.191 registration.py:141(make)
        1    0.000    0.000    0.000    0.000 registration.py:15(load)
        1    0.000    0.000    0.191    0.191 registration.py:49(make)
        1    0.000    0.000    0.191    0.191 registration.py:81(make)
     2481    0.001    0.000    0.002    0.000 registry.py:74(list)
     4962    0.004    0.000    0.009    0.000 registry.py:82(lookup)
      145    0.000    0.000    0.000    0.000 regularizers.py:297(get)
      742    0.001    0.000    0.725    0.001 resource_variable_ops.py:1233(_dense_var_to_tensor)
       56    0.000    0.000    0.054    0.001 resource_variable_ops.py:1335(__init__)
       56    0.003    0.000    0.054    0.001 resource_variable_ops.py:1436(_init_from_args)
       56    0.003    0.000    0.027    0.000 resource_variable_ops.py:146(_variable_handle_from_shape_and_dtype)
      742    0.001    0.000    0.726    0.001 resource_variable_ops.py:1824(_dense_var_to_tensor)
       56    0.000    0.000    0.028    0.000 resource_variable_ops.py:198(eager_safe_variable_handle)
       56    0.000    0.000    0.000    0.000 resource_variable_ops.py:268(__init__)
      742    0.001    0.000    0.003    0.000 resource_variable_ops.py:314(_maybe_set_handle_data)
      742    0.002    0.000    0.021    0.000 resource_variable_ops.py:326(variable_accessed)
       56    0.001    0.000    0.003    0.000 resource_variable_ops.py:337(__init__)
      486    0.000    0.000    0.000    0.000 resource_variable_ops.py:493(dtype)
       10    0.000    0.000    0.000    0.000 resource_variable_ops.py:513(shape)
      742    0.003    0.000    0.722    0.001 resource_variable_ops.py:545(value)
      798    0.000    0.000    0.000    0.000 resource_variable_ops.py:583(trainable)
      742    0.003    0.000    0.698    0.001 resource_variable_ops.py:631(_read_variable_op)
      742    0.003    0.000    0.671    0.001 resource_variable_ops.py:634(read_and_set_handle)
       56    0.000    0.000    0.000    0.000 resource_variable_ops.py:84(_set_handle_shapes_and_types)
        2    0.000    0.000    0.002    0.001 return_statements.py:111(visit_With)
        2    0.000    0.000    0.002    0.001 return_statements.py:151(visit_FunctionDef)
        8    0.000    0.000    0.000    0.000 return_statements.py:159(__init__)
        4    0.000    0.000    0.000    0.000 return_statements.py:172(__init__)
        2    0.000    0.000    0.000    0.000 return_statements.py:221(__init__)
        2    0.000    0.000    0.002    0.001 return_statements.py:225(visit_Return)
        9    0.000    0.000    0.000    0.000 return_statements.py:252(_postprocess_statement)
      4/2    0.000    0.000    0.004    0.002 return_statements.py:275(_visit_statement_block)
        2    0.000    0.000    0.004    0.002 return_statements.py:317(visit_With)
        2    0.000    0.000    0.011    0.005 return_statements.py:339(visit_FunctionDef)
        6    0.000    0.000    0.000    0.000 return_statements.py:37(__init__)
        2    0.000    0.000    0.013    0.006 return_statements.py:394(transform)
        2    0.000    0.000    0.000    0.000 return_statements.py:68(visit_Return)
        9    0.000    0.000    0.000    0.000 return_statements.py:72(_postprocess_statement)
      4/2    0.000    0.000    0.002    0.001 return_statements.py:91(_visit_statement_block)
        3    0.000    0.000    0.001    0.000 seeding.py:11(np_random)
        4    0.000    0.000    0.000    0.000 seeding.py:21(hash_seed)
        3    0.000    0.000    0.000    0.000 seeding.py:45(create_seed)
        7    0.000    0.000    0.000    0.000 seeding.py:69(_bigint_from_bytes)
        3    0.000    0.000    0.000    0.000 seeding.py:80(_int_list_from_bigint)
      294    0.000    0.000    0.001    0.000 shape_base.py:209(_arrays_for_stack_dispatcher)
      294    0.001    0.000    0.001    0.000 shape_base.py:348(_stack_dispatcher)
      294    0.003    0.000    0.024    0.000 shape_base.py:357(stack)
      294    0.001    0.000    0.002    0.000 shape_base.py:420(<listcomp>)
      294    0.001    0.000    0.001    0.000 shape_base.py:424(<setcomp>)
      294    0.001    0.000    0.001    0.000 shape_base.py:432(<listcomp>)
     2522    0.003    0.000    0.004    0.000 six.py:586(iteritems)
      128    0.000    0.000    0.825    0.006 smart_cond.py:27(smart_cond)
      246    0.000    0.000    0.002    0.000 smart_cond.py:62(smart_constant_value)
       36    0.003    0.000    0.003    0.000 socket.py:342(send)
        2    0.000    0.000    0.001    0.000 space.py:21(seed)
        2    0.000    0.000    0.001    0.000 space.py:9(__init__)
       35    0.000    0.000    0.000    0.000 special_values.py:90(retval)
       20    0.000    0.000    0.000    0.000 sre_parse.py:963(expand_template)
        3    0.000    0.000    0.000    0.000 std.py:1051(__repr__)
        2    0.000    0.000    0.000    0.000 std.py:1054(_comparable)
        2    0.000    0.000    0.000    0.000 std.py:1058(__hash__)
        2    0.000    0.000    0.004    0.002 std.py:1061(__iter__)
        1    0.000    0.000    0.001    0.001 std.py:1221(close)
        2    0.000    0.000    0.000    0.000 std.py:1238(fp_write)
        2    0.000    0.000    0.006    0.003 std.py:1275(refresh)
        4    0.000    0.000    0.004    0.001 std.py:1383(moveto)
        3    0.000    0.000    0.000    0.000 std.py:1388(format_dict)
        3    0.000    0.000    0.007    0.002 std.py:1403(display)
        3    0.000    0.000    0.000    0.000 std.py:146(__init__)
        3    0.000    0.000    0.000    0.000 std.py:153(__format__)
        5    0.000    0.000    0.000    0.000 std.py:226(format_interval)
        1    0.000    0.000    0.000    0.000 std.py:267(ema)
        1    0.000    0.000    0.000    0.000 std.py:286(status_printer)
        3    0.000    0.000    0.002    0.001 std.py:296(fp_write)
        3    0.000    0.000    0.002    0.001 std.py:302(print_status)
        3    0.000    0.000    0.000    0.000 std.py:309(format_meter)
        1    0.000    0.000    0.000    0.000 std.py:504(__new__)
        1    0.000    0.000    0.000    0.000 std.py:526(_get_free_pos)
        4    0.000    0.000    0.000    0.000 std.py:529(<genexpr>)
        1    0.000    0.000    0.000    0.000 std.py:533(_decr_instances)
        1    0.000    0.000    0.000    0.000 std.py:606(get_lock)
        1    0.000    0.000    0.003    0.003 std.py:772(__init__)
        6    0.000    0.000    0.000    0.000 std.py:88(acquire)
        6    0.000    0.000    0.000    0.000 std.py:92(release)
        4    0.000    0.000    0.000    0.000 std.py:96(__enter__)
        4    0.000    0.000    0.000    0.000 std.py:99(__exit__)
        1    1.256    1.256    3.151    3.151 summary_ops_v2.py:1062(run_metadata_graphs)
        1    0.000    0.000    0.004    0.004 summary_ops_v2.py:1157(trace_on)
        1    0.470    0.470    5.869    5.869 summary_ops_v2.py:1202(trace_export)
        2    0.000    0.000    0.000    0.000 summary_ops_v2.py:1258(trace_off)
        2    0.000    0.000    0.000    0.000 summary_ops_v2.py:573(summary_scope)
        1    0.000    0.000    0.000    0.000 summary_ops_v2.py:614(write)
      640    0.001    0.000    0.001    0.000 tape.py:117(stop_recording)
     1544    0.001    0.000    0.003    0.000 tape.py:144(record_operation)
    14994    0.011    0.000    0.028    0.000 tape.py:187(could_possibly_record)
      486    0.002    0.000    0.007    0.000 tape.py:73(variable_accessed)
        2    0.000    0.000    0.000    0.000 tempfile.py:236(_infer_return_type)
        2    0.000    0.000    0.000    0.000 tempfile.py:257(_sanitize_params)
        2    0.000    0.000    0.000    0.000 tempfile.py:285(rng)
        2    0.000    0.000    0.000    0.000 tempfile.py:296(__next__)
        2    0.000    0.000    0.000    0.000 tempfile.py:299(<listcomp>)
        2    0.000    0.000    0.000    0.000 tempfile.py:376(_get_candidate_names)
        2    0.000    0.000    0.000    0.000 tempfile.py:390(_mkstemp_inner)
        2    0.000    0.000    0.000    0.000 tempfile.py:430(gettempdir)
        2    0.000    0.000    0.000    0.000 tempfile.py:565(__init__)
        4    0.000    0.000    0.000    0.000 tempfile.py:580(close)
        2    0.000    0.000    0.000    0.000 tempfile.py:590(__del__)
        2    0.000    0.000    0.000    0.000 tempfile.py:608(__init__)
        2    0.000    0.000    0.000    0.000 tempfile.py:614(__getattr__)
        2    0.000    0.000    0.000    0.000 tempfile.py:622(func_wrapper)
        2    0.000    0.000    0.000    0.000 tempfile.py:635(__enter__)
        2    0.000    0.000    0.000    0.000 tempfile.py:641(__exit__)
        2    0.000    0.000    0.000    0.000 tempfile.py:646(close)
        2    0.000    0.000    0.000    0.000 tempfile.py:663(NamedTemporaryFile)
       28    0.000    0.000    0.000    0.000 templates.py:111(__init__)
       82    0.000    0.000    0.017    0.000 templates.py:129(_prepare_replacement)
       20    0.000    0.000    0.016    0.001 templates.py:146(visit_Expr)
        8    0.000    0.000    0.000    0.000 templates.py:154(visit_keyword)
      4/2    0.000    0.000    0.007    0.003 templates.py:172(visit_FunctionDef)
       22    0.000    0.000    0.001    0.000 templates.py:185(visit_Attribute)
      108    0.000    0.000    0.019    0.000 templates.py:197(visit_Name)
    81/70    0.000    0.000    0.000    0.000 templates.py:218(_convert_to_ast)
        8    0.000    0.000    0.000    0.000 templates.py:228(<listcomp>)
        4    0.000    0.000    0.000    0.000 templates.py:230(<genexpr>)
       18    0.000    0.000    0.034    0.002 templates.py:234(replace)
       18    0.000    0.000    0.008    0.000 templates.py:275(<listcomp>)
        6    0.000    0.000    0.005    0.001 templates.py:279(replace_as_expression)
       78    0.000    0.000    0.000    0.000 templates.py:43(__init__)
   140/52    0.000    0.000    0.001    0.000 templates.py:46(visit)
       70    0.000    0.000    0.000    0.000 templates.py:54(_apply_override)
        6    0.000    0.000    0.000    0.000 templates.py:58(visit_Attribute)
       10    0.000    0.000    0.000    0.000 templates.py:64(visit_Tuple)
       54    0.000    0.000    0.001    0.000 templates.py:72(visit_Name)
   615910    0.472    0.000    0.669    0.000 tensor_conversion_registry.py:114(get)
   480780    0.481    0.000    6.743    0.000 tensor_conversion_registry.py:50(_default_conversion_function)
       10    0.000    0.000    0.000    0.000 tensor_shape.py:1019(with_rank_at_least)
       56    0.000    0.000    0.000    0.000 tensor_shape.py:1057(is_compatible_with)
       40    0.000    0.000    0.001    0.000 tensor_shape.py:1119(most_specific_compatible_shape)
      144    0.000    0.000    0.001    0.000 tensor_shape.py:1149(is_fully_defined)
      448    0.000    0.000    0.000    0.000 tensor_shape.py:1152(<genexpr>)
      533    0.000    0.000    0.001    0.000 tensor_shape.py:1163(as_list)
      533    0.001    0.000    0.001    0.000 tensor_shape.py:1174(<listcomp>)
     1031    0.005    0.000    0.007    0.000 tensor_shape.py:1176(as_proto)
     1021    0.001    0.000    0.001    0.000 tensor_shape.py:1182(<listcomp>)
        2    0.000    0.000    0.000    0.000 tensor_shape.py:1186(__eq__)
      635    0.000    0.000    0.001    0.000 tensor_shape.py:1213(as_shape)
       64    0.000    0.000    0.000    0.000 tensor_shape.py:1221(unknown_shape)
       10    0.000    0.000    0.000    0.000 tensor_shape.py:130(dimension_at_index)
     3893    0.005    0.000    0.006    0.000 tensor_shape.py:185(__init__)
       62    0.000    0.000    0.000    0.000 tensor_shape.py:211(__eq__)
     6497    0.001    0.000    0.001    0.000 tensor_shape.py:243(value)
      419    0.000    0.000    0.001    0.000 tensor_shape.py:248(is_compatible_with)
      320    0.000    0.000    0.001    0.000 tensor_shape.py:264(assert_is_compatible_with)
      320    0.000    0.000    0.002    0.000 tensor_shape.py:278(merge_with)
     4728    0.003    0.000    0.010    0.000 tensor_shape.py:700(as_dimension)
     1655    0.004    0.000    0.017    0.000 tensor_shape.py:742(__init__)
     1528    0.002    0.000    0.011    0.000 tensor_shape.py:771(<listcomp>)
     2256    0.001    0.000    0.001    0.000 tensor_shape.py:773(_v2_behavior)
     1714    0.001    0.000    0.001    0.000 tensor_shape.py:802(rank)
      503    0.000    0.000    0.000    0.000 tensor_shape.py:809(dims)
      820    0.000    0.000    0.001    0.000 tensor_shape.py:821(ndims)
      295    0.000    0.000    0.000    0.000 tensor_shape.py:826(__len__)
       56    0.000    0.000    0.000    0.000 tensor_shape.py:832(__bool__)
      111    0.000    0.000    0.000    0.000 tensor_shape.py:839(__iter__)
      444    0.000    0.000    0.000    0.000 tensor_shape.py:845(<genexpr>)
     2219    0.003    0.000    0.004    0.000 tensor_shape.py:849(__getitem__)
       90    0.001    0.000    0.004    0.000 tensor_shape.py:909(merge_with)
       89    0.000    0.000    0.000    0.000 tensor_shape.py:97(dimension_value)
       90    0.000    0.000    0.000    0.000 tensor_shape.py:971(assert_same_rank)
       52    0.000    0.000    0.002    0.000 tensor_shape.py:999(with_rank)
       21    0.000    0.000    0.000    0.000 tensor_spec.py:39(__init__)
       21    0.000    0.000    0.000    0.000 tensor_spec.py:63(shape)
       15    0.000    0.000    0.001    0.000 tensor_util.py:1003(shape_tensor)
       93    0.000    0.000    0.002    0.000 tensor_util.py:1022(maybe_set_static_shape)
       48    0.000    0.000    0.000    0.000 tensor_util.py:185(GetFromNumpyDTypeDict)
       48    0.000    0.000    0.000    0.000 tensor_util.py:193(GetNumpyAppendFn)
  173/102    0.000    0.000    0.000    0.000 tensor_util.py:218(_GetDenseDimensions)
       72    0.000    0.000    0.001    0.000 tensor_util.py:262(inner)
       72    0.000    0.000    0.001    0.000 tensor_util.py:263(<listcomp>)
       30    0.000    0.000    0.000    0.000 tensor_util.py:275(_check_not_tensor)
       30    0.000    0.000    0.000    0.000 tensor_util.py:276(<listcomp>)
      102    0.000    0.000    0.001    0.000 tensor_util.py:304(_AssertCompatible)
      102    0.000    0.000    0.001    0.000 tensor_util.py:334(_is_array_like)
      102    0.003    0.000    0.010    0.000 tensor_util.py:355(make_tensor_proto)
        2    0.000    0.000    0.000    0.000 tensor_util.py:564(MakeNdarray)
        2    0.000    0.000    0.000    0.000 tensor_util.py:591(<listcomp>)
       32    0.000    0.000    0.001    0.000 tensor_util.py:669(_ConstantValue)
       42    0.000    0.000    0.001    0.000 tensor_util.py:797(constant_value)
        2    0.000    0.000    0.001    0.000 tensor_util.py:835(constant_value_as_shape)
        2    0.000    0.000    0.000    0.000 tensor_util.py:974(<listcomp>)
      586    0.001    0.000    0.003    0.000 tensor_util.py:978(is_tensor)
       18    0.000    0.000    0.000    0.000 textwrap.py:414(dedent)
      243    0.000    0.000    0.000    0.000 tf2.py:43(enabled)
     2119    0.004    0.000    0.042    0.000 tf_contextlib.py:25(contextmanager)
     1100    0.001    0.000    0.001    0.000 tf_decorator.py:114(_has_tf_decorator_attr)
       18    0.000    0.000    0.000    0.000 tf_decorator.py:128(rewrap)
      720    0.001    0.000    0.003    0.000 tf_decorator.py:200(unwrap)
     2149    0.005    0.000    0.006    0.000 tf_decorator.py:236(__init__)
      742    0.000    0.000    0.000    0.000 tf_decorator.py:262(decorated_target)
       18    0.000    0.000    0.000    0.000 tf_decorator.py:266(decorated_target)
       12    0.000    0.000    0.000    0.000 tf_decorator.py:278(decorator_argspec)
     2149    0.007    0.000    0.016    0.000 tf_decorator.py:67(make_decorator)
      145    0.000    0.000    0.008    0.000 tf_inspect.py:238(getfullargspec)
       60    0.001    0.000    0.005    0.000 tf_inspect.py:260(getcallargs)
       60    0.000    0.000    0.000    0.000 tf_inspect.py:283(<listcomp>)
       89    0.000    0.000    0.001    0.000 tf_inspect.py:335(getmodule)
        2    0.000    0.000    0.000    0.000 tf_inspect.py:350(getsourcefile)
        2    0.000    0.000    0.001    0.001 tf_inspect.py:355(getsourcelines)
      153    0.000    0.000    0.001    0.000 tf_inspect.py:365(isclass)
      111    0.000    0.000    0.000    0.000 tf_inspect.py:370(isfunction)
       17    0.000    0.000    0.000    0.000 tf_inspect.py:385(isgeneratorfunction)
      226    0.000    0.000    0.001    0.000 tf_inspect.py:390(ismethod)
        1    0.000    0.000    0.004    0.004 tf_logging.py:171(warn)
       36    0.000    0.000    0.000    0.000 tf_logging.py:210(vlog)
        1    0.000    0.000    0.000    0.000 tf_logging.py:45(_get_caller)
        1    0.000    0.000    0.000    0.000 tf_logging.py:75(_logger_find_caller)
       37    0.000    0.000    0.000    0.000 tf_logging.py:93(get_logger)
       35    0.000    0.000    0.002    0.000 tf_stack.py:105(__init__)
     1835    0.001    0.000    0.001    0.000 tf_stack.py:123(get_filtered_filenames)
     2514    0.004    0.000    0.079    0.000 tf_stack.py:131(extract_stack)
       70    0.000    0.000    0.000    0.000 tf_stack.py:53(__enter__)
       70    0.000    0.000    0.000    0.000 tf_stack.py:71(__exit__)
       35    0.000    0.000    0.000    0.000 tf_stack.py:83(reset)
       35    0.000    0.000    0.000    0.000 tf_stack.py:95(reset)
       20    0.001    0.000    0.003    0.000 tf_utils.py:102(get_reachable_from_inputs)
    72/36    0.000    0.000    0.001    0.000 tf_utils.py:157(map_structure_with_atomic)
       36    0.000    0.000    0.000    0.000 tf_utils.py:185(<listcomp>)
       36    0.000    0.000    0.001    0.000 tf_utils.py:193(convert_shapes)
      108    0.000    0.000    0.000    0.000 tf_utils.py:220(_is_shape_component)
       72    0.000    0.000    0.000    0.000 tf_utils.py:223(_is_atomic_shape)
       72    0.000    0.000    0.000    0.000 tf_utils.py:230(<genexpr>)
       36    0.000    0.000    0.000    0.000 tf_utils.py:234(_convert_shape)
      966    0.001    0.000    0.011    0.000 tf_utils.py:325(are_all_symbolic_tensors)
     1967    0.001    0.000    0.010    0.000 tf_utils.py:326(<genexpr>)
     1001    0.003    0.000    0.009    0.000 tf_utils.py:332(is_symbolic_tensor)
        3    0.000    0.000    0.000    0.000 tf_utils.py:411(assert_no_legacy_layers)
      128    0.000    0.000    0.826    0.006 tf_utils.py:42(smart_cond)
        3    0.000    0.000    0.000    0.000 tf_utils.py:425(<listcomp>)
       46    0.000    0.000    0.002    0.000 tf_utils.py:436(maybe_init_scope)
        3    0.000    0.000    0.000    0.000 tf_utils.py:475(get_tensor_spec)
      118    0.000    0.000    0.001    0.000 tf_utils.py:68(constant_value)
       44    0.000    0.000    0.000    0.000 threading.py:1062(_wait_for_tstate_lock)
        1    0.000    0.000    0.000    0.000 threading.py:1076(name)
       44    0.000    0.000    0.000    0.000 threading.py:1104(is_alive)
        1    0.000    0.000    0.000    0.000 threading.py:1230(current_thread)
       68    0.001    0.000    0.001    0.000 threading.py:215(__init__)
        8    0.000    0.000    0.000    0.000 threading.py:239(__enter__)
        8    0.000    0.000    0.000    0.000 threading.py:242(__exit__)
        6    0.000    0.000    0.000    0.000 threading.py:248(_release_save)
        6    0.000    0.000    0.000    0.000 threading.py:251(_acquire_restore)
     2490    0.002    0.000    0.003    0.000 threading.py:254(_is_owned)
        6    0.000    0.000    0.007    0.001 threading.py:263(wait)
     2484    0.006    0.000    0.009    0.000 threading.py:334(notify)
     2484    0.004    0.000    0.013    0.000 threading.py:357(notify_all)
        8    0.000    0.000    0.000    0.000 threading.py:498(__init__)
       45    0.000    0.000    0.000    0.000 threading.py:506(is_set)
        8    0.000    0.000    0.007    0.001 threading.py:533(wait)
       60    0.000    0.000    0.000    0.000 threading.py:74(RLock)
      294    0.002    0.000    0.216    0.001 time_limit.py:14(step)
        1    0.000    0.000    0.010    0.010 time_limit.py:23(reset)
        1    0.000    0.000    0.000    0.000 time_limit.py:5(__init__)
        1    0.000    0.000    0.000    0.000 tmp555ck6zk.py:2(<module>)
        1    0.000    0.000    0.000    0.000 tmp555ck6zk.py:2(create_converted_entity_factory)
        1    0.000    0.000    0.000    0.000 tmp555ck6zk.py:4(create_converted_entity)
        1    0.000    0.000    0.067    0.067 tmp555ck6zk.py:6(tf__initial_inference_compiled)
       17    0.000    0.000    0.000    0.000 tmpqiwkscxv.py:2(create_converted_entity_factory)
       17    0.000    0.000    0.000    0.000 tmpqiwkscxv.py:4(create_converted_entity)
       17    0.000    0.000    0.000    0.000 tmpqiwkscxv.py:6(tf____hash__)
        1    0.000    0.000    0.000    0.000 tmpwx_3xg_e.py:2(<module>)
       17    0.000    0.000    0.000    0.000 tmpwx_3xg_e.py:2(create_converted_entity_factory)
       17    0.000    0.000    0.000    0.000 tmpwx_3xg_e.py:4(create_converted_entity)
       17    0.001    0.000    1.224    0.072 tmpwx_3xg_e.py:6(tf__recurrent_inference_compiled)
      981    0.000    0.000    0.001    0.000 tokenize.py:152(_compile)
        2    0.000    0.000    0.000    0.000 tokenize.py:224(__init__)
        2    0.000    0.000    0.000    0.000 tokenize.py:243(untokenize)
        2    0.000    0.000    0.000    0.000 tokenize.py:280(compat)
        2    0.000    0.000    0.000    0.000 tokenize.py:317(untokenize)
     1059    0.003    0.000    0.006    0.000 tokenize.py:492(_tokenize)
       10    0.000    0.000    0.000    0.000 tokenize.py:739(generate_tokens)
     2481    0.003    0.000    0.021    0.000 tpu.py:210(tpu_replicated_input_resolver)
     4962    0.007    0.000    0.016    0.000 tpu.py:226(replace_with_unreplicated_resources)
     4547    0.007    0.000    0.007    0.000 traceable_stack.py:119(peek_objs)
     4547    0.001    0.000    0.001    0.000 traceable_stack.py:121(<genexpr>)
     4968    0.002    0.000    0.002    0.000 traceable_stack.py:123(peek_traceable_objs)
     7808    0.003    0.000    0.004    0.000 traceable_stack.py:127(__len__)
      116    0.000    0.000    0.000    0.000 traceable_stack.py:131(copy)
     1720    0.001    0.000    0.001    0.000 traceable_stack.py:83(__init__)
2210/2090    0.002    0.000    0.008    0.000 tracking.py:417(wrapped)
      164    0.000    0.000    0.000    0.000 tracking.py:77(__setattr__)
        3    0.000    0.000    0.010    0.003 training.py:166(__init__)
        3    0.000    0.000    0.001    0.000 training.py:352(_reset_compile_cache)
        3    0.000    0.000    0.000    0.000 training_utils.py:1901(get_input_shape_and_dtype)
        3    0.000    0.000    0.000    0.000 training_utils.py:1915(_is_graph_model)
        3    0.000    0.000    0.000    0.000 training_utils.py:1935(get_static_batch_size)
       64    0.000    0.000    0.000    0.000 transformer.py:100(__init__)
       24    0.000    0.000    0.000    0.000 transformer.py:108(__enter__)
       24    0.000    0.000    0.000    0.000 transformer.py:112(__exit__)
       84    0.000    0.000    0.000    0.000 transformer.py:115(enter)
       44    0.000    0.000    0.000    0.000 transformer.py:118(exit)
        2    0.000    0.000    0.000    0.000 transformer.py:121(stack)
      322    0.000    0.000    0.000    0.000 transformer.py:125(level)
      710    0.000    0.000    0.000    0.000 transformer.py:133(__iter__)
      108    0.000    0.000    0.000    0.000 transformer.py:136(__getattr__)
       51    0.000    0.000    0.000    0.000 transformer.py:139(__setattr__)
      110    0.000    0.000    0.000    0.000 transformer.py:175(__init__)
     1216    0.000    0.000    0.001    0.000 transformer.py:178(__getitem__)
      110    0.000    0.000    0.000    0.000 transformer.py:236(__init__)
   372/76    0.001    0.000    0.074    0.001 transformer.py:273(visit_block)
11164/110    0.025    0.000    0.185    0.002 transformer.py:408(visit)
        2    0.000    0.000    0.000    0.000 transformer.py:43(__init__)
       21    0.000    0.000    0.001    0.000 unparser.py:115(_Assign)
     15/9    0.000    0.000    0.000    0.000 unparser.py:14(interleave)
        6    0.000    0.000    0.000    0.000 unparser.py:141(_Return)
        2    0.000    0.000    0.000    0.000 unparser.py:223(_Raise)
        2    0.000    0.000    0.000    0.000 unparser.py:245(_Try)
        2    0.000    0.000    0.000    0.000 unparser.py:292(_ExceptHandler)
        2    0.000    0.000    0.001    0.001 unparser.py:32(__init__)
      6/2    0.000    0.000    0.001    0.000 unparser.py:346(_FunctionDef)
      6/2    0.000    0.000    0.001    0.000 unparser.py:352(__FunctionDef_helper)
       43    0.000    0.000    0.000    0.000 unparser.py:42(fill)
        2    0.000    0.000    0.001    0.000 unparser.py:422(_generic_With)
        2    0.000    0.000    0.001    0.000 unparser.py:435(_With)
        4    0.000    0.000    0.000    0.000 unparser.py:445(_Str)
      388    0.000    0.000    0.000    0.000 unparser.py:46(write)
       12    0.000    0.000    0.000    0.000 unparser.py:50(enter)
       83    0.000    0.000    0.000    0.000 unparser.py:520(_Name)
       17    0.000    0.000    0.000    0.000 unparser.py:523(_NameConstant)
       12    0.000    0.000    0.000    0.000 unparser.py:55(leave)
        8    0.000    0.000    0.000    0.000 unparser.py:555(_Num)
      5/1    0.000    0.000    0.000    0.000 unparser.py:570(_List)
        7    0.000    0.000    0.000    0.000 unparser.py:572(<lambda>)
    263/2    0.000    0.000    0.001    0.000 unparser.py:59(dispatch)
       15    0.000    0.000    0.000    0.000 unparser.py:651(_Tuple)
        8    0.000    0.000    0.000    0.000 unparser.py:658(<lambda>)
       28    0.000    0.000    0.000    0.000 unparser.py:706(_Attribute)
    19/15    0.000    0.000    0.000    0.000 unparser.py:716(_Call)
       11    0.000    0.000    0.000    0.000 unparser.py:772(_arg)
        6    0.000    0.000    0.000    0.000 unparser.py:779(_arguments)
       10    0.000    0.000    0.000    0.000 unparser.py:837(_keyword)
        2    0.000    0.000    0.000    0.000 unparser.py:859(_withitem)
        2    0.000    0.000    0.000    0.000 unparser.py:88(_Expr)
        8    0.000    0.000    0.000    0.000 unsupported_features_checker.py:32(visit_Attribute)
        2    0.000    0.000    0.001    0.000 unsupported_features_checker.py:60(verify)
        3    0.000    0.000    0.000    0.000 util.py:1070(__init__)
        3    0.000    0.000    0.000    0.000 util.py:1338(saver_with_op_caching)
        3    0.000    0.000    0.000    0.000 utils.py:134(__init__)
        3    0.000    0.000    0.000    0.000 utils.py:138(__format__)
        1    0.000    0.000    0.000    0.000 utils.py:151(__eq__)
        1    0.000    0.000    0.000    0.000 utils.py:188(_is_utf)
        1    0.000    0.000    0.000    0.000 utils.py:202(_supports_unicode)
        6    0.000    0.000    0.000    0.000 utils.py:209(_is_ascii)
        1    0.000    0.000    0.000    0.000 utils.py:218(_environ_cols_wrapper)
        1    0.000    0.000    0.000    0.000 utils.py:270(_environ_cols_linux)
        4    0.000    0.000    0.000    0.000 utils.py:288(_term_move_up)
       58    0.000    0.000    0.000    0.000 variable_scope.py:1167(use_resource)
      116    0.000    0.000    0.000    0.000 variable_scope.py:1203(set_use_resource)
       58    0.000    0.000    0.000    0.000 variable_scope.py:1421(get_variable_scope_store)
       58    0.000    0.000    0.001    0.000 variable_scope.py:1434(get_variable_scope)
       56    0.000    0.000    0.055    0.001 variable_scope.py:2558(default_variable_creator)
      112    0.000    0.000    0.000    0.000 variables.py:150(validate_synchronization_aggregation_trainable)
       56    0.000    0.000    0.055    0.001 variables.py:181(_variable_v1_call)
       56    0.000    0.000    0.055    0.001 variables.py:198(<lambda>)
   112/56    0.000    0.000    0.056    0.001 variables.py:257(__call__)
        3    0.000    0.000    0.000    0.000 version_utils.py:46(__new__)
       41    0.000    0.000    0.019    0.000 version_utils.py:55(__new__)
   114/44    0.018    0.000    0.018    0.000 version_utils.py:61(swap_class)
   137/85    0.000    0.000    0.014    0.000 version_utils.py:73(<genexpr>)
      426    0.001    0.000    0.002    0.000 weakref.py:102(__init__)
       11    0.000    0.000    0.000    0.000 weakref.py:109(remove)
      173    0.000    0.000    0.000    0.000 weakref.py:125(_commit_removals)
       55    0.000    0.000    0.000    0.000 weakref.py:165(__setitem__)
      228    0.000    0.000    0.001    0.000 weakref.py:208(items)
      426    0.000    0.000    0.000    0.000 weakref.py:288(update)
       55    0.000    0.000    0.000    0.000 weakref.py:334(__new__)
       55    0.000    0.000    0.000    0.000 weakref.py:339(__init__)
       47    0.000    0.000    0.000    0.000 weakref.py:354(__init__)
    46/40    0.001    0.000    0.053    0.001 weakref.py:356(remove)
       38    0.000    0.000    0.000    0.000 weakref.py:371(_commit_removals)
    15639    0.013    0.000    0.013    0.000 weakref.py:393(__getitem__)
      291    0.000    0.000    0.000    0.000 weakref.py:406(__setitem__)
     2329    0.002    0.000    0.003    0.000 weakref.py:428(get)
    15709    0.051    0.000    0.051    0.000 weakref.py:431(__contains__)
       76    0.000    0.000    0.000    0.000 weakref.py:438(items)
      123    0.000    0.000    0.000    0.000 weakref.py:480(pop)
       38    0.000    0.000    0.000    0.000 weakref.py:487(update)
    36163    0.026    0.000    0.026    0.000 {built-in method __new__ of type object at 0x9d12c0}
        4    0.000    0.000    0.000    0.000 {built-in method _hashlib.openssl_sha512}
        4    0.000    0.000    0.000    0.000 {built-in method _imp.acquire_lock}
        2    0.000    0.000    0.000    0.000 {built-in method _imp.extension_suffixes}
        8    0.000    0.000    0.000    0.000 {built-in method _imp.lock_held}
        4    0.000    0.000    0.000    0.000 {built-in method _imp.release_lock}
        7    0.000    0.000    0.000    0.000 {built-in method _struct.unpack}
      125    0.000    0.000    0.000    0.000 {built-in method _thread.allocate_lock}
     2590    0.001    0.000    0.001    0.000 {built-in method _thread.get_ident}
        5    0.000    0.000    0.000    0.000 {built-in method _weakref._remove_dead_weakref}
        2    0.000    0.000    0.000    0.000 {built-in method atexit.register}
       11    0.000    0.000    0.000    0.000 {built-in method builtins.abs}
   139469    0.087    0.000    0.197    0.000 {built-in method builtins.all}
     5753    0.005    0.000    0.015    0.000 {built-in method builtins.any}
     3009    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
       37    0.002    0.000    0.002    0.000 {built-in method builtins.compile}
       66    0.001    0.000    0.001    0.000 {built-in method builtins.dir}
       19    0.000    0.000    0.000    0.000 {built-in method builtins.divmod}
      3/1    0.000    0.000  190.543  190.543 {built-in method builtins.exec}
   262157    0.248    0.000    0.432    0.000 {built-in method builtins.getattr}
   155572    0.074    0.000    0.076    0.000 {built-in method builtins.hasattr}
79498/79328    0.039    0.000    0.045    0.000 {built-in method builtins.hash}
    12533    0.002    0.000    0.002    0.000 {built-in method builtins.id}
  6914761    1.723    0.000    2.721    0.000 {built-in method builtins.isinstance}
   616835    0.172    0.000    0.172    0.000 {built-in method builtins.issubclass}
     4952    0.001    0.000    0.001    0.000 {built-in method builtins.iter}
213609/213471    0.057    0.000    0.057    0.000 {built-in method builtins.len}
   131443    2.668    0.000  119.248    0.001 {built-in method builtins.max}
    71630    0.625    0.000   12.027    0.000 {built-in method builtins.min}
80698/65569    0.052    0.000    0.292    0.000 {built-in method builtins.next}
       63    0.000    0.000    0.000    0.000 {built-in method builtins.ord}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
       29    0.000    0.000    0.000    0.000 {built-in method builtins.repr}
    74904    0.039    0.000    0.063    0.000 {built-in method builtins.setattr}
    16448    0.030    0.000    0.031    0.000 {built-in method builtins.sorted}
    15288    0.019    0.000    0.020    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {built-in method fcntl.ioctl}
        3    0.000    0.000    0.000    0.000 {built-in method from_bytes}
        2    0.000    0.000    0.000    0.000 {built-in method io.open}
        2    0.000    0.000    0.000    0.000 {built-in method marshal.dumps}
    59976    0.037    0.000    0.037    0.000 {built-in method math.exp}
   227712    0.208    0.000    0.208    0.000 {built-in method math.log}
   227725    0.078    0.000    0.078    0.000 {built-in method math.sqrt}
      735    0.004    0.000    0.004    0.000 {built-in method numpy.array}
1642/1348    0.020    0.000    0.050    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      294    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.normalize_axis_index}
      297    0.001    0.000    0.001    0.000 {built-in method numpy.empty}
        2    0.000    0.000    0.000    0.000 {built-in method numpy.frombuffer}
       12    0.000    0.000    0.000    0.000 {built-in method numpy.geterrobj}
        6    0.000    0.000    0.000    0.000 {built-in method numpy.seterrobj}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
       24    0.000    0.000    0.000    0.000 {built-in method posix.fspath}
       14    0.000    0.000    0.000    0.000 {built-in method posix.getpid}
        4    0.000    0.000    0.000    0.000 {built-in method posix.open}
        2    0.000    0.000    0.000    0.000 {built-in method posix.replace}
       52    0.001    0.000    0.001    0.000 {built-in method posix.stat}
        6    0.000    0.000    0.000    0.000 {built-in method posix.urandom}
       36    0.000    0.000    0.000    0.000 {built-in method sys._getframe}
     1719    0.001    0.000    0.001    0.000 {built-in method sys.exc_info}
        2    0.000    0.000    0.000    0.000 {built-in method sys.getrecursionlimit}
       10    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tf_session.AddControlInput}
     2481    0.004    0.000    0.004    0.000 {built-in method tensorflow.python._pywrap_tf_session.GetOperationInputs}
       58    0.001    0.000    0.001    0.000 {built-in method tensorflow.python._pywrap_tf_session.SetAttr}
      782    0.007    0.000    0.007    0.000 {built-in method tensorflow.python._pywrap_tf_session.SetHandleShapeAndType}
       60    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tf_session.SetRequireShapeInferenceFns}
       20    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_AddInputList}
     2373    0.009    0.000    0.009    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_AddInput}
     1952    0.003    0.000    0.003    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_DeleteBuffer}
      538    0.088    0.000    0.088    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_DeleteFunction}
      544    0.228    0.000    0.228    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_DeleteGraph}
     2484    0.056    0.000    0.056    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_FinishOperation}
       58    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_FunctionToFunctionDef}
      175    0.038    0.000    0.038    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GetBuffer}
       40    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GraphCopyFunction}
       76    0.001    0.000    0.001    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GraphGetOpDef}
      561    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GraphGetTensorShapeHelper}
      782    0.003    0.000    0.003    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GraphSetOutputHandleShapesAndTypes_wrapper}
       75    0.001    0.000    0.001    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GraphSetTensorShape_wrapper}
       58    0.011    0.000    0.011    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_GraphToFunction_wrapper}
       58    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_NewBufferFromString}
     1894    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_NewBuffer}
       60    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_NewGraph}
     2484    0.025    0.000    0.025    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_NewOperation}
     1759    0.104    0.000    0.109    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationGetAttrValueProto}
       80    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationGetControlOutputs_wrapper}
     2443    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationName}
     2484    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationNumOutputs}
    32643    0.021    0.000    0.021    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationOpType}
      220    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationOutputConsumers_wrapper}
     3144    0.005    0.000    0.005    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_OperationOutputType}
     5101    0.021    0.000    0.021    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_SetAttrValueProto}
       30    0.001    0.000    0.001    0.000 {built-in method tensorflow.python._pywrap_tf_session.TF_TryEvaluateConstant_wrapper}
       58    0.005    0.000    0.005    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_ContextAddFunction}
        2    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_ContextDisableRunMetadata}
        1    1.149    1.149    1.149    1.149 {built-in method tensorflow.python._pywrap_tfe.TFE_ContextExportRunMetadata}
      538    0.541    0.001    0.541    0.001 {built-in method tensorflow.python._pywrap_tfe.TFE_ContextRemoveFunction}
        3    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_MonitoringBoolGaugeCellSet}
        3    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_MonitoringGetCellBoolGauge1}
    14994    0.125    0.000    0.125    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_EncodeArg}
    15288   12.214    0.001   12.214    0.001 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_Execute}
  1514521   48.281    0.000   48.281    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_FastPathExecute}
    16850    0.020    0.000    0.020    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeSetIsEmpty}
      320    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeSetIsStopped}
    14994    0.015    0.000    0.015    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeSetPossibleGradientTypes}
     1544    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeSetRecordOperation}
      152    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeSetRestartOnThread}
      152    0.000    0.000    0.000    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeSetStopOnThread}
      486    0.001    0.000    0.001    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_TapeVariableAccessed}
     3318    0.004    0.000    0.004    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_UID}
      136    0.001    0.000    0.001    0.000 {built-in method tensorflow.python._pywrap_utils.AssertSameStructure}
   203612    0.470    0.000    0.471    0.000 {built-in method tensorflow.python._pywrap_utils.Flatten}
    30816    0.013    0.000    0.013    0.000 {built-in method tensorflow.python._pywrap_utils.IsAttrs}
    30816    0.010    0.000    0.010    0.000 {built-in method tensorflow.python._pywrap_utils.IsCompositeTensor}
    15368    0.005    0.000    0.005    0.000 {built-in method tensorflow.python._pywrap_utils.IsMappingView}
    15404    0.005    0.000    0.005    0.000 {built-in method tensorflow.python._pywrap_utils.IsMapping}
    15954    0.009    0.000    0.009    0.000 {built-in method tensorflow.python._pywrap_utils.IsMutableMapping}
    30816    0.066    0.000    0.066    0.000 {built-in method tensorflow.python._pywrap_utils.IsNamedtuple}
    90561    0.049    0.000    0.049    0.000 {built-in method tensorflow.python._pywrap_utils.IsSequenceOrComposite}
     5181    0.002    0.000    0.002    0.000 {built-in method tensorflow.python._pywrap_utils.IsSequence}
    30816    0.009    0.000    0.009    0.000 {built-in method tensorflow.python._pywrap_utils.IsTypeSpec}
     2514    0.072    0.000    0.074    0.000 {built-in method tensorflow.python._tf_stack.extract_stack}
        6    0.000    0.000    0.000    0.000 {built-in method time.time}
     4968    0.017    0.000    0.017    0.000 {method 'ByteSize' of 'google.protobuf.pyext._message.CMessage' objects}
     5256    0.009    0.000    0.009    0.000 {method 'CopyFrom' of 'google.protobuf.pyext._message.CMessage' objects}
     8276    0.006    0.000    0.006    0.000 {method 'HasField' of 'google.protobuf.pyext._message.CMessage' objects}
      175    1.070    0.006    1.070    0.006 {method 'ParseFromString' of 'google.protobuf.pyext._message.CMessage' objects}
     5942    1.839    0.000    1.839    0.000 {method 'SerializeToString' of 'google.protobuf.pyext._message.CMessage' objects}
      434    0.000    0.000    0.000    0.000 {method 'SetInParent' of 'google.protobuf.pyext._message.CMessage' objects}
       40    0.000    0.000    0.000    0.000 {method 'WhichOneof' of 'google.protobuf.pyext._message.CMessage' objects}
        2    0.000    0.000    0.000    0.000 {method '__enter__' of '_io._IOBase' objects}
        8    0.000    0.000    0.000    0.000 {method '__enter__' of '_thread.lock' objects}
        2    0.000    0.000    0.000    0.000 {method '__exit__' of '_io._IOBase' objects}
        8    0.000    0.000    0.000    0.000 {method '__exit__' of '_thread.lock' objects}
  1999051    0.430    0.000    0.430    0.000 {method '_datatype_enum' of 'tensorflow.python.framework.ops.EagerTensor' objects}
   480366   59.494    0.000   59.494    0.000 {method '_numpy_internal' of 'tensorflow.python.framework.ops.EagerTensor' objects}
      474    0.001    0.000    0.001    0.000 {method '_shape_tuple' of 'tensorflow.python.framework.ops.EagerTensor' objects}
        6    0.000    0.000    0.000    0.000 {method 'acquire' of '_multiprocessing.SemLock' objects}
        8    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.RLock' objects}
    22514    0.022    0.000    0.022    0.000 {method 'acquire' of '_thread.lock' objects}
     7818    0.003    0.000    0.005    0.000 {method 'add' of 'set' objects}
       42    0.000    0.000    0.000    0.000 {method 'append' of 'collections.deque' objects}
       56    0.000    0.000    0.000    0.000 {method 'append' of 'google.protobuf.pyext._message.RepeatedCompositeContainer' objects}
   398801    0.068    0.000    0.068    0.000 {method 'append' of 'list' objects}
       22    0.000    0.000    0.000    0.000 {method 'astype' of 'numpy.ndarray' objects}
       16    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
      294    0.028    0.000    0.039    0.000 {method 'choice' of 'numpy.random.mtrand.RandomState' objects}
        2    0.000    0.000    0.000    0.000 {method 'close' of '_io.TextIOWrapper' objects}
       61    0.000    0.000    0.000    0.000 {method 'copy' of 'dict' objects}
    15006    0.052    0.000    0.052    0.000 {method 'copy' of 'numpy.ndarray' objects}
       38    0.000    0.000    0.000    0.000 {method 'count' of 'list' objects}
       98    0.000    0.000    0.000    0.000 {method 'decode' of 'bytes' objects}
        1    0.000    0.000    0.000    0.000 {method 'difference' of 'set' objects}
        4    0.000    0.000    0.000    0.000 {method 'digest' of '_hashlib.HASH' objects}
      294    0.009    0.000    0.016    0.000 {method 'dirichlet' of 'numpy.random.mtrand.RandomState' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
      427    0.000    0.000    0.000    0.000 {method 'discard' of 'set' objects}
     5874    0.003    0.000    0.003    0.000 {method 'encode' of 'str' objects}
    45065    0.024    0.000    0.024    0.000 {method 'endswith' of 'str' objects}
        6    0.000    0.000    0.000    0.000 {method 'extend' of 'bytearray' objects}
      129    0.000    0.000    0.000    0.000 {method 'extend' of 'collections.deque' objects}
       20    0.000    0.000    0.000    0.000 {method 'extend' of 'google.protobuf.pyext._message.RepeatedCompositeContainer' objects}
      414    0.000    0.000    0.000    0.000 {method 'extend' of 'google.protobuf.pyext._message.RepeatedScalarContainer' objects}
     4803    0.002    0.000    0.002    0.000 {method 'extend' of 'list' objects}
      931    0.000    0.000    0.000    0.000 {method 'find' of 'str' objects}
       18    0.000    0.000    0.000    0.000 {method 'findall' of '_sre.SRE_Pattern' objects}
        2    0.000    0.000    0.000    0.000 {method 'flush' of '_io._IOBase' objects}
      165    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
      296    0.000    0.000    0.000    0.000 {method 'from_address' of '_ctypes.PyCArrayType' objects}
   164547    0.074    0.000    0.083    0.000 {method 'get' of 'dict' objects}
       23    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
        2    0.000    0.000    0.000    0.000 {method 'getvalue' of '_io.StringIO' objects}
       66    0.000    0.000    0.000    0.000 {method 'group' of '_sre.SRE_Match' objects}
      200    0.000    0.000    0.000    0.000 {method 'insert' of 'list' objects}
       20    0.000    0.000    0.000    0.000 {method 'intersection' of 'set' objects}
       10    0.000    0.000    0.000    0.000 {method 'isdigit' of 'str' objects}
     1337    0.000    0.000    0.000    0.000 {method 'isidentifier' of 'str' objects}
       35    0.000    0.000    0.000    0.000 {method 'issubset' of 'set' objects}
    81879    0.023    0.000    0.023    0.000 {method 'items' of 'dict' objects}
       77    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
    19534    0.006    0.000    0.006    0.000 {method 'keys' of 'dict' objects}
     3040    0.001    0.000    0.001    0.000 {method 'lower' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
     5617    0.009    0.000    0.009    0.000 {method 'match' of '_sre.SRE_Pattern' objects}
      888    0.001    0.000    0.001    0.000 {method 'newbyteorder' of 'numpy.dtype' objects}
     9619    0.002    0.000    0.002    0.000 {method 'pop' of 'dict' objects}
    54795    0.029    0.000    0.029    0.000 {method 'pop' of 'list' objects}
    19900    0.042    0.000    0.042    0.000 {method 'popitem' of 'collections.OrderedDict' objects}
      129    0.000    0.000    0.000    0.000 {method 'popleft' of 'collections.deque' objects}
      294    0.005    0.000    0.005    0.000 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
       48    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 {method 'read' of '_io.FileIO' objects}
       77    0.000    0.000    0.000    0.000 {method 'readline' of '_io.StringIO' objects}
     1346    0.014    0.000    0.015    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        6    0.000    0.000    0.000    0.000 {method 'release' of '_multiprocessing.SemLock' objects}
        8    0.000    0.000    0.000    0.000 {method 'release' of '_thread.RLock' objects}
    19968    0.006    0.000    0.006    0.000 {method 'release' of '_thread.lock' objects}
      964    0.000    0.000    0.000    0.000 {method 'remove' of 'set' objects}
       50    0.000    0.000    0.000    0.000 {method 'replace' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        6    0.000    0.000    0.000    0.000 {method 'rfind' of 'str' objects}
       16    0.000    0.000    0.000    0.000 {method 'rpartition' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {method 'rsplit' of 'str' objects}
       16    0.000    0.000    0.000    0.000 {method 'rstrip' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'search' of '_sre.SRE_Pattern' objects}
        3    0.000    0.000    0.000    0.000 {method 'seed' of 'numpy.random.mtrand.RandomState' objects}
       38    0.000    0.000    0.000    0.000 {method 'setdefault' of 'dict' objects}
      100    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
      981    0.000    0.000    0.000    0.000 {method 'span' of '_sre.SRE_Match' objects}
       24    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
     8159    0.002    0.000    0.002    0.000 {method 'startswith' of 'str' objects}
       19    0.000    0.000    0.000    0.000 {method 'strip' of 'str' objects}
      111    0.001    0.000    0.001    0.000 {method 'sub' of '_sre.SRE_Pattern' objects}
      294    0.001    0.000    0.003    0.000 {method 'sum' of 'numpy.ndarray' objects}
     1719    0.001    0.000    0.003    0.000 {method 'throw' of 'generator' objects}
        4    0.000    0.000    0.000    0.000 {method 'to_bytes' of 'int' objects}
       54    0.000    0.000    0.000    0.000 {method 'tostring' of 'numpy.ndarray' objects}
     2227    0.001    0.000    0.001    0.000 {method 'update' of 'dict' objects}
     9163    0.002    0.000    0.003    0.000 {method 'update' of 'set' objects}
      321    0.000    0.000    0.000    0.000 {method 'upper' of 'str' objects}
      272    0.000    0.000    0.000    0.000 {method 'values' of 'collections.OrderedDict' objects}
    18050    0.006    0.000    0.006    0.000 {method 'values' of 'dict' objects}
      145    0.000    0.000    0.000    0.000 {method 'values' of 'mappingproxy' objects}
        2    0.000    0.000    0.000    0.000 {method 'write' of '_io.FileIO' objects}
      431    0.000    0.000    0.000    0.000 {method 'write' of '_io.StringIO' objects}
        2    0.000    0.000    0.000    0.000 {method 'write' of '_io.TextIOWrapper' objects}
      295    0.010    0.000    0.010    0.000 {resize}
       20    0.000    0.000    0.000    0.000 {tensorflow.python.framework.fast_tensor_util.AppendBoolArrayToTensorProto}
       28    0.000    0.000    0.000    0.000 {tensorflow.python.framework.fast_tensor_util.AppendFloat32ArrayToTensorProto}"""

In [ ]:
import pandas as pd

# df_trace = pd.DataFrame([x.split()[:5]+[''.join(x.split()[5:])] for x in trace.split('\n')[1:]], columns=['ncalls','tottime','percall','cumtime','percall_cum','filename'])
df_trace = pd.read_csv('trace.txt')
df_trace = df_trace.astype({'tottime':'float','percall':'float','cumtime':'float','percall_cum':'float'})

In [50]:
pd.set_option('max_colwidth', 800)

In [ ]:
df_trace.sort_values(by=['tottime'],ascending=False).head(25)

In [10]:
import pdb
pdb.pm()

> /Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/ray/worker.py(1515)get()
-> raise value.as_instanceof_cause()


(Pdb)  u


> /Users/tfrizza/Documents/robotic_learning/muzero/muzero/train.py(44)train_network()
-> batch = ray.get(ray_id)


(Pdb)  ll


 27  	def train_network(config: MuZeroConfig, storage: SharedStorage,
 28  	                                    replay_buffer: ReplayBuffer):
 29  	#     while len(replay_buffer.buffer)==0: time.sleep(1)
 30  	    while ray.get(replay_buffer.get_buffer_size.remote()) < 1:
 31  	        time.sleep(1)
 32  	    network = Network_FC(config) if config.model_type == "fc" else Network_CNN(config) # Network()
 33  	    learning_rate = config.lr_init * config.lr_decay_rate**(
 34  	#           get_global_step() / config.lr_decay_steps)
 35  	            1)
 36  	    optimizer = Adam(learning_rate, config.momentum)
 37  	
 38  	    progbar = tf.keras.utils.Progbar(config.training_steps, verbose=1, stateful_metrics=None, unit_name='step')
 39  	
 40  	    for i in range(config.training_steps): #tqdm(range(config.training_steps), desc='Training iter'):
 41  	        if i % config.checkpoint_interval == 0:
 42  	            storage.save_network(i, network)
 43  	        ray_id = replay_buffer.samp

(Pdb)  q


In [12]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs